In [ ]:
import numpy as np
import os
import h5py
import time
from scipy.spatial import KDTree
import json
from compare_eis import compare_eis


# --- Path and recording setup ---
dat_path = "/Volumes/Lab/Users/alexth/axolotl/201703151_data001.dat"
n_channels = 512
dtype = np.int16

# --- Get total number of samples ---
file_size_bytes = os.path.getsize(dat_path)
total_samples = file_size_bytes // (np.dtype(dtype).itemsize * n_channels)

# --- Load entire file into RAM as int16 ---
raw_data = np.fromfile(dat_path, dtype=dtype, count=total_samples * n_channels)
raw_data = raw_data.reshape((total_samples, n_channels))  # shape: [T, C]


# --- Parameters ---
n_channels = 512
dtype = 'int16'
max_units = 1500
amplitude_threshold = 15
window = (-20, 60)
peak_window = 30
total_samples=36_000_000
fit_offsets = (-5, 10)

do_pursuit = 0


h5_in_path = '/Volumes/Lab/Users/alexth/axolotl/201703151_kilosort_data001_spike_times.h5'  # from MATLAB export, to get EI positions
h5_out_path = '/Volumes/Lab/Users/alexth/axolotl/results_pipeline_0528.h5' # where to save data

debug_folder = "/Volumes/Lab/Users/alexth/axolotl/debug"

with h5py.File(h5_in_path, 'r') as f:
    # Load electrode positions
    ei_positions = f['/ei_positions'][:].T  # shape becomes [512 x 2]
    ks_vision_ids = f['/vision_ids'][:]  # shape: (N_units,)

import axolotl_utils_ram
import importlib
importlib.reload(axolotl_utils_ram)

save_path = "/Volumes/Lab/Users/alexth/axolotl/201703151_data001_baseline_and_artifacts.json"

if os.path.exists(save_path):
    print(f"Loading baselines")
    with open(save_path, 'r') as f:
        data = json.load(f)
    baselines = np.array(data['baselines'], dtype=np.float32)
else:
    print(f"Computing baselines")
    baselines = axolotl_utils_ram.compute_baselines_int16(raw_data, segment_len=100_000) # shape (512, 360)

    with open(save_path, 'w') as f:
        json.dump({
            'baselines': baselines.tolist(),
        }, f)


# get KS EIs
ks_ei_path = '/Volumes/Lab/Users/alexth/axolotl/ks_eis_subset.h5'
ks_templates = {}
ks_n_spikes = {}

with h5py.File(ks_ei_path, 'r') as f:
    for k in f.keys():
        unit_id = int(k.split('_')[1])-1
        ks_templates[unit_id] = f[k][:]
        ks_n_spikes[unit_id] = f[k].attrs.get('n_spikes', -1)  # fallback if missing

ks_unit_ids = list(ks_templates.keys())
ks_ei_stack = np.stack([ks_templates[k] for k in ks_unit_ids], axis=0)  # [N x 512 x 81]

unit_id = 0

print(f"\n=== Starting unit {unit_id} ===")

while True:


    start_time = time.time()

    # could cache scores on channels to pre-identify next one
    ref_channel = axolotl_utils_ram.find_dominant_channel_ram(
            raw_data = raw_data,
            segment_len = 100_000,
            n_segments = 10,
            peak_window = 30,
            top_k_neg = 20,
            top_k_events = 5,
            seed = 42
        )

    threshold, spike_times = axolotl_utils_ram.estimate_spike_threshold_ram(
        raw_data=raw_data,
        ref_channel=ref_channel,
        window = 30,
        total_samples_to_read = total_samples,
        refractory = 30,
        top_n = 100
    )

    print(f"Channel: {ref_channel}, Threshold: {-threshold:.1f}, Initial spikes: {len(spike_times)}")

    snips, valid_spike_times = axolotl_utils_ram.extract_snippets_ram(
        raw_data=raw_data,
        spike_times=spike_times,
        window=window,
        selected_channels=np.arange(n_channels)
    )

    ei = np.mean(snips, axis=2)
    ei -= ei[:, :5].mean(axis=1, keepdims=True)

    spikes_for_plot_pre = valid_spike_times

    # Step 6–7: Cluster and select dominant unit
    clusters_pre, pcs_pre, labels_pre, sim_matrix_pre, cluster_eis_pre  = axolotl_utils_ram.cluster_spike_waveforms(snips, ei, k_start=3,return_debug=True)

    ei, spikes_idx, selected_channels, selected_cluster_index_pre = axolotl_utils_ram.select_cluster_with_largest_waveform(clusters_pre, ref_channel)

    spikes_init = spike_times[spikes_idx]

    if do_pursuit:
        (
        spikes,
        mean_score,
        valid_score,
        mean_scores_at_spikes,
        valid_scores_at_spikes,
        mean_thresh,
        valid_thresh
        ) = axolotl_utils_ram.ei_pursuit_ram(
            raw_data=raw_data,
            spikes=spikes_init,                     # absolute sample times
            ei_template=ei,                    # EI from selected cluster
            save_prefix='/Volumes/Lab/Users/alexth/axolotl/ei_scan_unit0',  # set uniquely per unit
            alignment_offset = -window[0],
            fit_percentile = 40,                # how many (percentile) spikes to take to fit Gaussian for threshold determination (left-hand side of already found spikes)
            sigma_thresh = 5.0,                  # how many Gaussian sigmas to take for threshold
            return_debug=True, 

        )
    else:
        spikes = spikes_init
        mean_score=None
        valid_score=None
        mean_scores_at_spikes=spikes
        valid_scores_at_spikes=None
        mean_thresh=None
        valid_thresh=None

    # Step 9a: Extract full snippets from final spike times

    snips_ref_channel, valid_spike_times = axolotl_utils_ram.extract_snippets_ram(
        raw_data=raw_data,
        spike_times=spikes,
        selected_channels=np.array([ref_channel]),
        window=window,
    )

    snips_ref_channel = snips_ref_channel.transpose(2, 0, 1)


    lags = axolotl_utils_ram.estimate_lags_by_xcorr_ram(
        snippets=snips_ref_channel,                # shape [N x C x T]
        peak_channel_idx=0,                 # 0 because the only channel that gets passed is the referent channel
        window=(-5, 10),                  # optional, relative to peak
        max_lag=6,                        # optional, max xcorr shift
    )

    spikes = spikes+lags

    snips_full, valid_spike_times = axolotl_utils_ram.extract_snippets_ram(
        raw_data=raw_data,
        spike_times=spikes,
        selected_channels=np.arange(n_channels),
        window=window,
    )


    segment_len = 100_000
    snips_baselined = snips_full.copy()  # shape (n_channels, 81, N)
    n_channels, snip_len, n_spikes = snips_baselined.shape

    # Determine segment index for each spike
    segment_indices = spikes // segment_len  # shape: (n_spikes,)

    # Loop through channels and subtract baseline per spike
    for ch in range(n_channels):
        snips_baselined[ch, :, :] -= baselines[ch, segment_indices][None, :]

    # Extract baseline-subtracted waveforms for ref_channel
    ref_snips = snips_baselined[ref_channel, :, :]  # shape: (81, N)

    # Mean waveform over all spikes
    ref_mean = ref_snips.mean(axis=1)  # shape: (81,)
    # Negative peak (should be near index 20)
    ref_peak_amp = np.abs(ref_mean[-window[0]])  # scalar

    # Threshold at 0.66× of mean waveform peak
    threshold_ampl = 0.66 * ref_peak_amp

    # Get all actual spike values at sample 20
    spike_amplitudes = np.abs(ref_snips[20, :])  # shape: (N,)

    # Flag bad spikes: too small
    bad_inds = np.where(spike_amplitudes < threshold_ampl)[0]

    # Create mask to keep only good spikes
    keep_mask = np.ones(spike_amplitudes.shape[0], dtype=bool)
    keep_mask[bad_inds] = False

    # --- Extract bad spike traces for plotting
    bad_spike_traces = snips_baselined[ref_channel, :, bad_inds]  # shape: (n_bad, T)

    # Get original traces for bad_spike_traces
    snips_bad = axolotl_utils_ram.extract_snippets_single_channel(
        dat_path='/Volumes/Lab/Users/alexth/axolotl/201703151_data001.dat',
        spike_times=spikes[bad_inds],
        ref_channel=ref_channel,
        window=window,
        n_channels=512,
        dtype='int16'
    )

    segment_indices = spikes[bad_inds] // segment_len  # shape: (n_spikes,)
    snips_bad[0, :, :] -= baselines[ref_channel, segment_indices][None, :]


    # Apply to real data and snips_baselined
    snips_baselined = snips_baselined[:, :, keep_mask]
    good_mean_trace = np.mean(snips_baselined[ref_channel, :, :], axis=1)
    snips_full = snips_full[:, :, keep_mask]
    valid_spike_times = valid_spike_times[keep_mask]
    spikes = spikes[keep_mask]

    spikes_for_plot_post = spikes


    # Step 9b: Recluster - choose k. snips_full is all channels, baselined - relevant cahnnels will be subselected in the function.


    if len(spikes)<100:
        pcs_post = np.zeros((1, 2))                    # shape: (N_spikes, 2 PCs)
        labels_post = np.array([0])                    # just one fake cluster label
        sim_matrix_post = np.zeros((1, 1))             # fake 1×1 similarity matrix
        ei_clusters_post = [np.zeros((512, 81))]       # fake EI for the “post” cluster
        selected_index_post = 0                        # only one cluster, so index is 0
        cluster_eis_post = [np.zeros((512, 81))]       # same dummy EI
        spikes_for_plot_post = np.array([0])           # placeholder spike time
        spike_counts_post = [len(snips)]               # use actual number of spikes
        matches = []                                # no matches
        # `snips_baselined` is [C x T x N]
        # We only subtract on the referent channel to avoid distortion
        template_fallback = np.mean(snips_baselined[ref_channel], axis=1)  # shape: (T,)
        residuals_fallback = snips_baselined[ref_channel] - template_fallback[:, None]  # shape: (T, N)

        # Assume residuals_fallback is (T, N) from previous step (template-subtracted waveforms)
        # Transpose to match expected shape: (n_spikes, snip_len)
        # force key and lookup to match normal case: np.int64
        ref_channel = np.int64(ref_channel)
        selected_channels = np.array([ref_channel], dtype=np.int64)
        residuals_per_channel = {
            ref_channel: residuals_fallback.T.astype(np.int16)
        }

    else:
        clusters_post, pcs_post, labels_post, sim_matrix_post, cluster_eis_post  = axolotl_utils_ram.cluster_spike_waveforms(snips=snips_baselined, ei=ei, k_start=2,return_debug=True)

        # Step 9c: choose the best cluster - choose similarity threshold. EI is all channels, baselined
        ei, final_spike_inds, selected_channels, selected_cluster_index_post = axolotl_utils_ram.select_cluster_by_ei_similarity_ram(clusters=clusters_post,reference_ei=ei,similarity_threshold=0.95)


        spikes = spikes[final_spike_inds]  # convert to absolute spike times
        snips_baselined = snips_baselined[:,:,final_spike_inds] # cut only the ones that survived

        p2p_threshold = 30
        ei_p2p = ei.max(axis=1) - ei.min(axis=1)
        selected_channels = np.where(ei_p2p > p2p_threshold)[0]
        selected_channels = selected_channels[np.argsort(ei_p2p[selected_channels])[::-1]]

        #print("reclustered pursuit\n")

        # check for matching KS units
        results = []
        lag = 20
        ks_sim_threshold = 0.75

        # Run comparison
        sim = compare_eis(ks_ei_stack, ei, lag).squeeze() # shape: (num_KS_units,)
        matches = [
            {
                "unit_id": ks_unit_ids[i],
                "vision_id": int(ks_vision_ids[ks_unit_ids[i]].item()),
                "similarity": float(sim[i]),
                "n_spikes": int(ks_n_spikes[ks_unit_ids[i]])
            }
            for i in np.where(sim > ks_sim_threshold)[0]
        ]



    # DIAGNOSTIC PLOTS

    axolotl_utils_ram.plot_unit_diagnostics(
        output_path=debug_folder,
        unit_id=unit_id,

        # --- From first call to cluster_spike_waveforms
        pcs_pre=pcs_pre,
        labels_pre=labels_pre,
        sim_matrix_pre=sim_matrix_pre,
        cluster_eis_pre = cluster_eis_pre,
        spikes_for_plot_pre = spikes_for_plot_pre,

        # --- From ei_pursuit
        mean_score=mean_score,
        valid_score=valid_score,
        mean_scores_at_spikes=mean_scores_at_spikes,
        valid_scores_at_spikes=valid_scores_at_spikes,
        mean_thresh=mean_thresh,
        valid_thresh=valid_thresh,

        # --- Lag estimation and bad spike filtering
        lags=lags,
        bad_spike_traces=bad_spike_traces,  # shape: (n_bad, T)
        good_mean_trace=good_mean_trace,
        threshold_ampl=-threshold_ampl,
        ref_channel=ref_channel,
        snips_bad=snips_bad,

        # --- From second clustering
        pcs_post=pcs_post,
        labels_post=labels_post,
        sim_matrix_post=sim_matrix_post,
        cluster_eis_post = cluster_eis_post,
        spikes_for_plot_post = spikes_for_plot_post,

        # --- For axis labels etc.
        window=(-20, 60),

        ei_positions=ei_positions,
        selected_channels_count=len(selected_channels),

        spikes = spikes, 
        orig_threshold = threshold,
        ks_matches = matches
    )


    # Step 10: Save unit metadata
    try:
        with h5py.File(h5_out_path, 'a') as h5:
            group = h5.require_group(f'unit_{unit_id}')

            for name, data in [
                ('spike_times', spikes.astype(np.int32)),
                ('ei', ei.astype(np.float32)), # EI is already baselined
                ('selected_channels', selected_channels.astype(np.int32))
            ]:
                if name in group:
                    del group[name]
                group.create_dataset(name, data=data)

            group.attrs['peak_channel'] = int(np.argmax(np.ptp(ei, axis=1)))
            # group.create_dataset('spike_times', data=spikes.astype(np.int32))
            # group.create_dataset('ei', data=ei.astype(np.float32))
            # group.create_dataset('selected_channels', data=selected_channels.astype(np.int32))
            # group.attrs['peak_channel'] = int(np.argmax(np.ptp(ei, axis=1)))

        #print(f"Exported unit_{unit_id} with {len(spikes)} spikes.")

    except KeyboardInterrupt:
        print("\nKeyboard interrupt detected — exiting safely before write completes.")

    except Exception as e:
        print(f"\nUnexpected error while saving unit_{unit_id}: {e}")



    if len(spikes)>=100:
        snips_full = snips_full[np.ix_(selected_channels, np.arange(snips_full.shape[1]), final_spike_inds)]
        snips_full = snips_full.transpose(2, 0, 1) # [C × T × N] → [N × C × T]

            # --- Setup ---
        residuals_per_channel = {}
        cluster_ids_per_channel = {}
        scale_factors_per_channel = {}

        for ch_idx, ch in enumerate(selected_channels):
            # Slice data for this channel
            ch_snips = snips_full[:, ch_idx, :]  # shape: (n_spikes, snip_len)
            ch_baselines = baselines[ch, :]    # shape: (n_segments,)

            # Subtract PCA cluster means
            residuals, cluster_ids, scale_factors = axolotl_utils_ram.subtract_pca_cluster_means_ram(
                snippets=ch_snips,
                baselines=ch_baselines,
                spike_times=spikes,
                segment_len=100_000,  # must match what was used to generate baselines
                n_clusters=5,
                offset_window=(-10,40)
            )

            # Store results
            residuals_per_channel[ch] = residuals
            cluster_ids_per_channel[ch] = cluster_ids
            scale_factors_per_channel[ch] = scale_factors
    else:
        
        # We only subtract on the referent channel to avoid distortion
        template_fallback = np.mean(snips_baselined[ref_channel], axis=1)  # shape: (T,)
        residuals_fallback = snips_baselined[ref_channel] - template_fallback[:, None]  # shape: (T, N)

        # Assume residuals_fallback is (T, N) from previous step (template-subtracted waveforms)
        # Transpose to match expected shape: (n_spikes, snip_len)
        # force key and lookup to match normal case: np.int64
        ref_channel = np.int64(ref_channel)
        selected_channels = np.array([ref_channel], dtype=np.int64)
        residuals_per_channel = {
            ref_channel: residuals_fallback.T.astype(np.int16)
        }

    # end_time = time.time()
    # elapsed = end_time - start_time 
    # print(f"Finished preprocessing, starting edits. Elapsed: {elapsed:.1f} seconds.")
    # Step 12: edit raw data
    write_locs = spikes + window[0]
    axolotl_utils_ram.apply_residuals(
        raw_data=raw_data,
        dat_path = '/Volumes/Lab/Users/alexth/axolotl/201703151_data001_sub.dat',
        residual_snips_per_channel=residuals_per_channel,
        write_locs=write_locs,
        selected_channels=selected_channels,
        total_samples=raw_data.shape[0],
        dtype = np.int16,
        n_channels = n_channels,
        is_ram=True,
        is_disk=False
    )
    end_time = time.time()
    elapsed = end_time - start_time
    print(f"Processed unit {unit_id} with {len(spikes)} final spikes in {elapsed:.1f} seconds.\n")


    # Step 13: Repeat until done
    unit_id += 1
    if unit_id >= max_units:
        print("Reached unit limit.")
        break



In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(25, 4))
plt.plot(raw_data[:5000, 39])
plt.xlabel('Sample')
plt.ylabel('Amplitude')
plt.title('Channel 39: First 5,000 samples')
plt.grid(True)
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# --- Parameters ---
dat_path = '/Volumes/Lab/Users/alexth/axolotl/201703151_data001_sub.dat'  # ← replace with actual path
n_channels = 512
channel = 39
n_samples = 5000
dtype = np.int16

# --- Read from disk ---
with open(dat_path, 'rb') as f:
    # Read first 5000 timepoints (i.e., 5000 * n_channels values)
    raw = np.fromfile(f, dtype=dtype, count=n_samples * n_channels)
    raw = raw.reshape((n_samples, n_channels))  # [time, channel]

# --- Plot ---
plt.figure(figsize=(25, 4))
plt.plot(raw[:5000, 39])
plt.xlabel('Sample')
plt.ylabel('Amplitude')
plt.title('Channel 39: First 5,000 samples')
plt.grid(True)
plt.show()


### Test - development

In [1]:
import numpy as np
import os
import h5py
import time
from scipy.spatial import KDTree
import json
from compare_eis import compare_eis


# --- Path and recording setup ---
dat_path = "/Volumes/Lab/Users/alexth/axolotl/201703151_data001.dat"
n_channels = 512
dtype = np.int16

# --- Get total number of samples ---
file_size_bytes = os.path.getsize(dat_path)
total_samples = file_size_bytes // (np.dtype(dtype).itemsize * n_channels)

# --- Load entire file into RAM as int16 ---
raw_data = np.fromfile(dat_path, dtype=dtype, count=total_samples * n_channels)
raw_data = raw_data.reshape((total_samples, n_channels))  # shape: [T, C]


# --- Parameters ---
n_channels = 512
dtype = 'int16'
max_units = 1500
amplitude_threshold = 15
window = (-20, 60)
peak_window = 30
total_samples=36_000_000
fit_offsets = (-5, 10)

do_pursuit = 0


h5_in_path = '/Volumes/Lab/Users/alexth/axolotl/201703151_kilosort_data001_spike_times.h5'  # from MATLAB export, to get EI positions


with h5py.File(h5_in_path, 'r') as f:
    # Load electrode positions
    ei_positions = f['/ei_positions'][:].T  # shape becomes [512 x 2]
    ks_vision_ids = f['/vision_ids'][:]  # shape: (N_units,)

import axolotl_utils_ram
import importlib
importlib.reload(axolotl_utils_ram)

baseline_path = "/Volumes/Lab/Users/alexth/axolotl/201703151_data001_baseline_derivative_20k.json"

segment_len = 20_000
if os.path.exists(baseline_path):
    print(f"Loading baselines")
    with open(baseline_path, 'r') as f:
        data = json.load(f)
    baselines = np.array(data['baselines'], dtype=np.float32)
else:
    print(f"Computing baselines")
    baselines = axolotl_utils_ram.compute_baselines_int16_deriv_robust(raw_data, segment_len=segment_len, diff_thresh=10, trim_fraction=0.15) # shape (512, 360)

    with open(baseline_path, 'w') as f:
        json.dump({
            'baselines': baselines.tolist(),
        }, f)

print("subtracting baselines")

axolotl_utils_ram.subtract_segment_baselines_int16(raw_data=raw_data,
                                     baselines_f32=baselines,
                                     segment_len=segment_len) 


# get KS EIs
ks_ei_path = '/Volumes/Lab/Users/alexth/axolotl/ks_eis_subset.h5'
ks_templates = {}
ks_n_spikes = {}

with h5py.File(ks_ei_path, 'r') as f:
    for k in f.keys():
        unit_id = int(k.split('_')[1])-1
        ks_templates[unit_id] = f[k][:]
        ks_n_spikes[unit_id] = f[k].attrs.get('n_spikes', -1)  # fallback if missing

ks_unit_ids = list(ks_templates.keys())
ks_ei_stack = np.stack([ks_templates[k] for k in ks_unit_ids], axis=0)  # [N x 512 x 81]


Matplotlib is building the font cache; this may take a moment.


Loading baselines
subtracting baselines


In [3]:
unit_id = 0
debug_folder = "/Volumes/Lab/Users/alexth/axolotl/debug/0621"
h5_out_path = '/Volumes/Lab/Users/alexth/axolotl/results_pipeline_0621.h5' # where to save data

In [4]:
import axolotl_utils_ram
import importlib
importlib.reload(axolotl_utils_ram)
from diptest import diptest
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from scipy.signal import find_peaks
from sklearn.cluster import KMeans
from scipy.stats import chi2
from sklearn.neighbors import LocalOutlierFactor


# import sys

# class Tee:
#     def __init__(self, *files, flush_every=10):
#         self.files = files
#         self.line_count = 0
#         self.flush_every = flush_every

#     def write(self, obj):
#         for f in self.files:
#             f.write(obj)
#         # Count lines
#         self.line_count += obj.count('\n')
#         if self.line_count >= self.flush_every:
#             self.flush()
#             self.line_count = 0  # Reset counter

#     def flush(self):
#         for f in self.files:
#             f.flush()



# # Create the log file
# log_file = open("/Volumes/Lab/Users/alexth/axolotl/debug/0616/processing_log.txt", "w")

# # Redirect stdout to both notebook and file
# sys.stdout = Tee(sys.__stdout__, log_file, flush_every=10)

# unit_id = 1004


# last_ref_channel = None
# remaining_candidates = []

do_second_clustering = False

if 'rejected_spike_log' not in globals():
    rejected_spike_log = {
        'times'   : [],   # absolute sample indices
        'channel' : [],   # ref_channel for each rejected snip
        'unit_id' : []    # which unit’s refinement produced the reject
    }

# ref_channel = 51

max_units = 1500


window = (-40, 80)

window_post_lag = (-20,60)

ax_ei_list = []

thresholds = np.zeros(511, dtype=float)
spike_length = np.zeros(511, dtype=int)
for ref_channel in range(0,511):
    threshold, spike_times = axolotl_utils_ram.estimate_spike_threshold_ram(
        raw_data=raw_data,
        ref_channel=ref_channel,
        window = 30,
        total_samples_to_read = 5_000_000,
        refractory = 10,
        top_n = 100,
        threshold_scale = 0.5
    )
    thresholds[ref_channel] = threshold
    spike_length[ref_channel] = len(spike_times)
 
   
sorted_indices = np.argsort(-thresholds)  # ascending (min to max, most negative to least negative)

# for idx in sorted_indices:
#     print(f"Channel {idx}, threshold {-thresholds[idx]:.2f}, spike N {spike_length[idx]}")

In [ ]:

import axolotl_utils_ram
import importlib
importlib.reload(axolotl_utils_ram)

# if 1:
while True:

    print(f"\n=== Starting unit {unit_id} ===\n")

    start_time = time.time()

    ref_channel = np.argmax(thresholds)

    # --------------------------------------------------------------------
    # 1.  Find all threshold crossings on ref_channel
    # --------------------------------------------------------------------

    threshold, spike_times = axolotl_utils_ram.estimate_spike_threshold_ram(
        raw_data=raw_data,
        ref_channel=ref_channel,
        window = 30,
        total_samples_to_read = total_samples,
        refractory = 10,
        top_n = 100,
        threshold_scale = 0.5
    )
    

    elapsed_thresholding = time.time() - start_time 
    print(f"Threshold: {elapsed_thresholding:.1f} s")

    print(f"Channel: {ref_channel}, Threshold: {-threshold:.1f}, Initial spikes: {len(spike_times)}")

    # --------------------------------------------------------------------
    # 2.  Extract data snippets for each spike
    # --------------------------------------------------------------------

    snips_baselined, valid_spike_times = axolotl_utils_ram.extract_snippets_fast_ram(
        raw_data=raw_data,
        spike_times=spike_times,
        window=window,
        selected_channels=np.arange(n_channels)
    )
    
    elapsed_snippet = time.time() - start_time 
    print(f"Snippet: {elapsed_snippet-elapsed_thresholding:.1f} s")

    # compute EI
    ei = np.mean(snips_baselined, axis=2)

    spikes_for_plot_pre = valid_spike_times

    # --------------------------------------------------------------------
    # 3.  Run PCA and k-means with adaptive k on extracted spikes
    # --------------------------------------------------------------------

    import axolotl_utils_ram
    import importlib
    importlib.reload(axolotl_utils_ram)

    k_start = min(5, 3 + (len(spike_times) - 1) // 3000)

    clusters_pre, pcs_pre, labels_pre, sim_matrix_pre, n_bad_channels_pre, cluster_eis_pre, cluster_to_merged_group_pre  = axolotl_utils_ram.cluster_spike_waveforms(snips_baselined, ei, k_start=k_start,return_debug=True)

    elapsed_clustering = time.time() - start_time 
    print(f"Clustering: {elapsed_clustering-elapsed_snippet:.1f} s, with k={k_start}")

    # --------------------------------------------------------------------
    # 4.  Select cluster with the largest negative amplitude for processing. Future: use all that are fully extracted?
    # --------------------------------------------------------------------

    ei, spikes_idx, selected_channels, selected_cluster_index_pre = axolotl_utils_ram.select_cluster_with_largest_waveform(clusters_pre, ref_channel)

    # ei shape: [channels, time]
    # Compute p2p for each channel
    p2p = ei.max(axis=1) - ei.min(axis=1)

    # Find channel with largest p2p - make sure it is the current ref_channel. If not, implement skip in the future - spikes should be sorted on Main
    best_channel = np.argmax(p2p)
    best_p2p = p2p[best_channel]
    if best_channel != ref_channel:
        print(f"Largest p2p at channel {best_channel}, p2p = {best_p2p:.2f}, IS NOT ref channel {ref_channel}")

    # indices of all clusters that were merged from original k-means - for diagnostic plot
    contributing_original_ids_pre = [
        orig_id for orig_id, merged_idx in cluster_to_merged_group_pre.items()
        if merged_idx == selected_cluster_index_pre
    ]
    contributing_original_ids_pre = np.array(contributing_original_ids_pre)

    spikes = spike_times[spikes_idx]

    # --------------------------------------------------------------------
    # 5.  Compute lags on the main channel to align the spikes (by somatic waveform)
    # --------------------------------------------------------------------

    snips_ref_channel = snips_baselined[ref_channel,:,spikes_idx]
    snips_ref_channel = snips_ref_channel[:, np.newaxis, :]


    lags = axolotl_utils_ram.estimate_lags_by_xcorr_ram(
        snippets=snips_ref_channel,                # shape [N x C x T]
        peak_channel_idx=0,                 # 0 because the only channel that gets passed is the referent channel
        window=(-5, 10),                  # optional, relative to peak
        max_lag=6,                        # optional, max xcorr shift
    )

    spikes = spikes+lags

    # --------------------------------------------------------------------
    # 6.  Re-extract the data snippets for aligned spikes. Works faster than shifting the snippets...
    # --------------------------------------------------------------------

    snips_full, valid_spike_times = axolotl_utils_ram.extract_snippets_fast_ram(
        raw_data=raw_data,
        spike_times=spikes,
        selected_channels=np.arange(n_channels),
        window=window,
    )

    elapsed_snippet_extraction = time.time() - start_time 
    print(f"Snippet: {elapsed_snippet_extraction-elapsed_clustering:.1f} s")


    # --------------------------------------------------------------------
    # 7.  Reanalyze ONLY the main channel for outliers - either randomly co-opted false spikes, or collisions/distorted WFs.
    # --------------------------------------------------------------------
    ref_snips     = snips_full[ref_channel, :, :].copy()
    ref_snips     = ref_snips.T          # [N × 81]
    wave_window   = slice(15, 40)                                 # focus on peak
    waveforms     = ref_snips[:, wave_window]                     # [N × 25]

    # --------------------------------------------------------------------
    # 7.1.  PCA, 2 components
    # --------------------------------------------------------------------
    pcs_raw = PCA(n_components=2, svd_solver='full').fit_transform(waveforms)  # [N × 2]

    # --------------------------------------------------------------------
    # 7.2.  Hartigan’s dip test on rotated projections
    # --------------------------------------------------------------------
    angle_step = 10
    angles = np.deg2rad(np.arange(0, 180, angle_step))

    best_p = 1.0
    best_proj = None
    for theta in angles:
        proj = pcs_raw[:, 0] * np.cos(theta) + pcs_raw[:, 1] * np.sin(theta)
        _, p = diptest(proj)
        if p < best_p:
            best_p, best_proj = p, proj

    discard_inds_bimodal = np.empty(0, dtype=int)

    if best_p < 0.05:                                   # suspected bimodality
        # histogram → first two peaks as initial centroids
        hist, bin_edges = np.histogram(best_proj, bins=30)
        peaks, _ = find_peaks(hist)
        if len(peaks) >= 2:
            centroids = bin_edges[peaks[:2]].reshape(-1, 1)
            km = KMeans(n_clusters=2, init=centroids, n_init=1, random_state=42)
            labels = km.fit_predict(best_proj[:, None])

            # keep larger lobe, discard smaller one
            counts = np.bincount(labels)
            keep_label = counts.argmax()
            discard_inds_bimodal = np.where(labels != keep_label)[0]

    # ---------------------------------------------------------------
    # 7.3.  Discard spikes that are too far or in a smaller split-up cluster. Currently, just zero them.
    # ---------------------------------------------------------------

    if discard_inds_bimodal.size:
        keep_mask  = np.ones(len(pcs_raw), dtype=bool)
        keep_mask[discard_inds_bimodal] = False
        snips_full = snips_full[:, :, keep_mask]
        valid_spike_times = valid_spike_times[keep_mask]
        spikes = spikes[keep_mask]
        waveforms     = waveforms[keep_mask, :]
        pcs_raw   = PCA(n_components=2, svd_solver='full').fit_transform(waveforms)
        print(f"   Split the final cluster according to Hartigan test! Started with {len(keep_mask)} spikes, ended with {len(spikes)}")

    # ---------------------------------------------------------------
    # 7.4.  z-score PCs, Mahalanobis (actually equivalent to euclidean after z-scoring in 2D)
    # ---------------------------------------------------------------

    pcs_z = StandardScaler().fit_transform(pcs_raw)
    pcs = pcs_z
    
    d2 = np.sum(pcs_z**2, axis=1)


    thr_vis  = chi2.ppf(0.999,  df=2)     # illustration
    thr_cut  = chi2.ppf(0.9999, df=2)     # discard

    final_outlier_inds      = np.where(d2 > thr_vis)[0]
    final_outlier_inds_max  = np.where(d2 > thr_cut)[0]

    # --------------------------------------------------------------------
    # 7.5.  Local Outlier Factor
    # --------------------------------------------------------------------
        # ----------------- build robust inlier core -------------------------
    core_mask = d2 < chi2.ppf(0.999, df=2)       # Mahalanobis core (~0.1 % trimmed)

    # ----------------- semi-supervised LOF on the core ------------------
    lof = LocalOutlierFactor(n_neighbors=20, novelty=True)
    lof.fit(pcs[core_mask])                    # train ONLY on good spikes
    lof_pred   = lof.predict(pcs)              # −1 = outlier w.r.t. core
    lof_scores = lof.negative_outlier_factor_      # more negative = more outlying
    lof_inds   = np.where(lof_pred == -1)[0]

    final_outlier_inds_max  = np.union1d(final_outlier_inds_max, lof_inds)


    # ------------------------------------------------------------------
    # 7.6.  Handle rejected spikes: find the spike region with signal on the ref_channel
    # ------------------------------------------------------------------
    pre, post        = window                     # snippet definition
    n_channels, snip_len, n_spikes = snips_full.shape
    accepted_mask    = np.ones(n_spikes, dtype=bool)
    accepted_mask[final_outlier_inds_max] = False
    accepted_inds    = np.where(accepted_mask)[0]


    if accepted_inds.size:                         # normal case
        mean_ref = snips_full[ref_channel, :, accepted_inds].copy()
    else:                                          # degenerate (all rejected)
        mean_ref = snips_full[ref_channel, :, :].copy()

    mean_ref = mean_ref.T
    mean_ref = np.mean(mean_ref, axis=1)
    mean_ref = mean_ref-mean_ref[0]

    abs_ref = np.abs(mean_ref)

    # --- thresholds in μV ------------------------------------------------
    max_abs = abs_ref.max()
    low_thr = 0.01 * max_abs
    high_thr = 0.1 * max_abs

    # first index where |WF| > 5 µV  (fall back to 0)
    try:
        idx_start = int(np.where(abs_ref > low_thr)[0][0])
    except IndexError:
        idx_start = 0

    # last index where |WF| > 20 µV  (fall back to end)
    cands = np.where(abs_ref > high_thr)[0]
    idx_end = int(cands[-1]) if cands.size else snip_len - 1

    print(f"Template masking for reject spikes: {idx_start} to {idx_end}")


    # convert waveform indices → sample offsets relative to spike centre
    # global_sample = spike_time + (wave_idx + pre)
    offset_start = idx_start + pre            # pre negative
    offset_end   = idx_end   + pre            # inclusive
    if offset_start > offset_end:             # safety
        offset_start, offset_end = pre, post

    # ------------------------------------------------------------------
    # 7.7.  Stash rejects
    # ------------------------------------------------------------------

    reject_times = spikes[final_outlier_inds_max]
    n_rej        = len(reject_times)

    for t in reject_times:
        rejected_spike_log['times']  .append(int(t))
        rejected_spike_log['channel'].append(ref_channel)
        rejected_spike_log['unit_id'].append(unit_id)


    # ------------------------------------------------------------------
    # 7.8.  Collect data snippets of rejected spikes to display in diagnostics plot
    # ------------------------------------------------------------------

    bad_inds = final_outlier_inds_max #outlier_inds

    # Create mask to keep only good spikes
    keep_mask = np.ones(spikes.shape[0], dtype=bool)
    keep_mask[bad_inds] = False

    # --- Extract bad spike traces for plotting
    bad_spike_traces_easy = snips_full[ref_channel, :, final_outlier_inds]  # shape: (n_bad, T)
    bad_spike_traces = snips_full[ref_channel, :, bad_inds]  # shape: (n_bad, T)

    # ------------------------------------------------------------------
    # 8.  Collect surviving spikes and final spike times
    # ------------------------------------------------------------------

    snips_full = snips_full[:, :, keep_mask]
    good_mean_trace = np.mean(snips_full[ref_channel, :, :], axis=1)
    valid_spike_times = valid_spike_times[keep_mask]
    spikes = spikes[keep_mask]
    spikes_for_plot_post = spikes

    final_spike_inds = np.where(keep_mask)[0]

    elapsed_bad = time.time() - start_time 
    print(f"Distortion handling: {elapsed_bad - elapsed_snippet_extraction:.1f} s, stashed {len(bad_inds)} spikes")

    # ------------------------------------------------------------------
    # 9.  Compute EI, selected_channels, cut snippets for records and for subtraction
    # ------------------------------------------------------------------

    final_ei = np.mean(snips_full, axis=2)

    p2p_threshold = 30
    ei_p2p = final_ei.max(axis=1) - final_ei.min(axis=1)
    selected_channels = np.where(ei_p2p > p2p_threshold)[0]
    selected_channels = selected_channels[np.argsort(ei_p2p[selected_channels])[::-1]]
    snips_full = snips_full[selected_channels, :, :]
    
    # ------------------------------------------------------------------
    # 10.  Find KS and Axolotl matches by simple EI
    # ------------------------------------------------------------------

    # check for matching KS units
    results = []
    lag = 20
    ks_sim_threshold = 0.75

    # Run comparison
    sim = compare_eis(ks_ei_stack, final_ei[:,20:101], lag).squeeze() # shape: (num_KS_units,)
    matches = [
        {
            "unit_id": ks_unit_ids[i],
            "vision_id": int(ks_vision_ids[ks_unit_ids[i]].item()),
            "similarity": float(sim[i]),
            "n_spikes": int(ks_n_spikes[ks_unit_ids[i]])
        }
        for i in np.where(sim > ks_sim_threshold)[0]
    ]
    # Find and print best match regardless of threshold
    best_idx = np.argmax(sim)
    best_ks_unit_id = ks_unit_ids[best_idx]
    best_ks_vision_id = int(ks_vision_ids[best_ks_unit_id].item())
    best_sim = sim[best_idx]

    print(f"Closest Kilosort unit {best_ks_vision_id} with similarity {best_sim:.3f}")

    elapsed_post_clustering= time.time() - start_time 
    print(f"Matches: {elapsed_post_clustering-elapsed_bad:.1f} s")


    if len(ax_ei_list) > 0:
        ax_ei_stack = np.stack(ax_ei_list, axis=0)
        sim = compare_eis(ax_ei_stack, final_ei, lag).reshape(-1) 
        best_idx = np.argmax(sim)
        best_sim = sim[best_idx]

        print(f"Closest Axolotl unit {best_idx} with similarity {best_sim:.3f}")

    # Append current final_ei
    ax_ei_list.append(final_ei)

    # ------------------------------------------------------------------
    # 11.  DIAGNOSTIC PLOTS
    # ------------------------------------------------------------------

    import axolotl_utils_ram
    import importlib
    importlib.reload(axolotl_utils_ram)

    axolotl_utils_ram.plot_unit_diagnostics_single_cluster(
        output_path=debug_folder,
        unit_id=unit_id,

        # --- From first call to cluster_spike_waveforms
        pcs_pre=pcs_pre,
        labels_pre=labels_pre,
        sim_matrix_pre=sim_matrix_pre,
        cluster_eis_pre = cluster_eis_pre,
        spikes_for_plot_pre = spikes_for_plot_pre,
        n_bad_channels_pre = n_bad_channels_pre,
        contributing_original_ids_pre = contributing_original_ids_pre,

        # --- Lag estimation and bad spike filtering
        lags=lags,
        bad_spike_traces=bad_spike_traces,  # shape: (n_bad, T)
        bad_spike_traces_easy=bad_spike_traces_easy,  # shape: (n_bad, T)
        pcs = pcs,
        outlier_inds_easy = final_outlier_inds,
        outlier_inds = final_outlier_inds_max,
        good_mean_trace=good_mean_trace,
        ref_channel=ref_channel,

        final_ei = final_ei,

        # --- For axis labels etc.

        ei_positions=ei_positions,

        spikes = spikes, 
        orig_threshold = threshold,
        ks_matches = matches
    )


    elapsed_diagnostic= time.time() - start_time 
    print(f"Diagnostics: {elapsed_diagnostic-elapsed_post_clustering:.1f} s")


    # ------------------------------------------------------------------
    # 12.  Save unit metadata
    # ------------------------------------------------------------------

    try:
        with h5py.File(h5_out_path, 'a') as h5:
            group = h5.require_group(f'unit_{unit_id}')

            for name, data in [
                ('spike_times', spikes.astype(np.int32)),
                ('ei', final_ei.astype(np.float32)), # EI is already baselined
                ('selected_channels', selected_channels.astype(np.int32))
            ]:
                if name in group:
                    del group[name]
                group.create_dataset(name, data=data)

            group.attrs['peak_channel'] = int(np.argmax(np.ptp(ei, axis=1)))

        #print(f"Exported unit_{unit_id} with {len(spikes)} spikes.")

    except KeyboardInterrupt:
        print("\nKeyboard interrupt detected — exiting safely before write completes.")

    except Exception as e:
        print(f"\nUnexpected error while saving unit_{unit_id}: {e}")


    elapsed_saving = time.time() - start_time 
    print(f"Saving: {elapsed_saving-elapsed_diagnostic:.1f} s")


    # ------------------------------------------------------------------
    # 13.  Compute residuals on selected channels
    # ------------------------------------------------------------------


    if len(spikes)>=100:
        
        snips_full = snips_full.transpose(2, 0, 1) # [C × T × N] → [N × C × T]

            # --- Setup ---
        residuals_per_channel = {}
        cluster_ids_per_channel = {}
        scale_factors_per_channel = {}

        for ch_idx, ch in enumerate(selected_channels):
            # Slice data for this channel
            ch_snips = snips_full[:, ch_idx, :]  # shape: (n_spikes, snip_len)
            ch_baselines = baselines[ch, :]*0    # shape: (n_segments,)

            # Subtract PCA cluster means
            residuals, cluster_ids, scale_factors = axolotl_utils_ram.subtract_pca_cluster_means_ram(
                snippets=ch_snips,
                baselines=ch_baselines,
                spike_times=spikes,
                segment_len=segment_len,  # must match what was used to generate baselines
                n_clusters=5,
                offset_window=(-10,40)
            )

            # Store results
            residuals_per_channel[ch] = residuals
            cluster_ids_per_channel[ch] = cluster_ids
            scale_factors_per_channel[ch] = scale_factors
    else:
        
        idx_in_selected = np.where(selected_channels == ref_channel)[0]
        idx_in_selected = idx_in_selected[0]

        # We only subtract on the referent channel to avoid distortion
        template_fallback = np.mean(snips_full[idx_in_selected], axis=1)  # shape: (T,)
        residuals_fallback = snips_full[idx_in_selected] - template_fallback[:, None]  # shape: (T, N)

        # Assume residuals_fallback is (T, N) from previous step (template-subtracted waveforms)
        # Transpose to match expected shape: (n_spikes, snip_len)
        # force key and lookup to match normal case: np.int64
        ref_channel = np.int64(ref_channel)
        selected_channels = np.array([ref_channel], dtype=np.int64)
        residuals_per_channel = {
            ref_channel: residuals_fallback.T.astype(np.int16)
        }


    elapsed_residual = time.time() - start_time 
    print(f"Residual: {elapsed_residual-elapsed_saving:.1f} s")


    # ------------------------------------------------------------------
    # 14.  Edit raw data - can do in RAM, file, or both. Usually RAM only for testing.
    # ------------------------------------------------------------------

    write_locs = spikes + window[0]
    axolotl_utils_ram.apply_residuals(
        raw_data=raw_data,
        dat_path = '/Volumes/Lab/Users/alexth/axolotl/201703151_data001_sub.dat',
        residual_snips_per_channel=residuals_per_channel,
        write_locs=write_locs,
        selected_channels=selected_channels,
        total_samples=raw_data.shape[0],
        dtype = np.int16,
        n_channels = n_channels,
        is_ram=True,
        is_disk=False
    )

    # ------------------------------------------------------------------
    # 15.  Zero out rejected spikes - not ideal, maybe nan or masking would work better. 
    # ------------------------------------------------------------------

    for t in reject_times:
        # ----   overwrite ref-channel samples with baseline --------
        t0 = int(t + offset_start)
        t1 = int(t + offset_end + 1)           # slice end is non-inclusive
        # bounds check
        if t0 < 0:            t0 = 0
        if t1 > raw_data.shape[0]:
            t1 = raw_data.shape[0]
        raw_data[t0:t1, ref_channel] = 0

    # ------------------------------------------------------------------
    # 16.  Re-estimate thresholds
    # ------------------------------------------------------------------

    cnt = 0
    for new_ref_channel in range(0, 511):
        neg_amp = -np.min(final_ei[new_ref_channel, :])
        existing_threshold = thresholds[new_ref_channel]

        if neg_amp >= 0.9 * abs(existing_threshold):
            # EI amplitude close enough to threshold → recompute
            threshold, spike_times = axolotl_utils_ram.estimate_spike_threshold_ram(
                raw_data=raw_data,
                ref_channel=new_ref_channel,
                window=30,
                total_samples_to_read=5_000_000,
                refractory=10,
                top_n=100,
                threshold_scale=0.5
            )
            thresholds[new_ref_channel] = threshold
            cnt = cnt+1

    print(f"Recomputed threshold on {cnt} channels")
    end_time = time.time()
    elapsed = end_time - start_time

    print(f"Subtraction + threshold: {elapsed -elapsed_residual:.1f} s")
    print(f"Processed unit {unit_id} with {len(spikes)} final spikes in {elapsed:.1f} seconds.\n")
    # print(f"timing. threshold: {elapsed_thresholding:.1f}, cluster: {elapsed_clustering-elapsed_thresholding:.1f},lags: {elapsed_lags-elapsed_thresholding:.1f},post: {elapsed_post_clustering-elapsed_lags:.1f},diag: {elapsed_diagnostic-elapsed_post_clustering:.1f},.\n")


    # Step 13: Repeat until done
    unit_id += 1
    # if unit_id >= max_units:
    #     print("Reached unit limit.")
    #     break

# sys.stdout = sys.__stdout__
# log_file.close()




### Generative model test - DEVELOPMENT

In [ ]:
import axolotl_utils_ram
import importlib
importlib.reload(axolotl_utils_ram)


units = axolotl_utils_ram.load_units_from_h5(h5_out_path)

unit_ids = sorted(units.keys())
selected_channels_list = [units[uid]['selected_channels'] for uid in unit_ids]

In [ ]:
target_channel = 413
max_unit_id = 92

units_with_413 = [
    unit_id for unit_id, data in units.items()
    if unit_id <= max_unit_id and target_channel in data['selected_channels']
]

print(f"Units with channel {target_channel} and unit_id ≤ {max_unit_id}: {units_with_413}")

# Get all unit IDs (sorted, to ensure order is clear)
unit_ids = sorted(units.keys())
num_units = len(unit_ids)

print(num_units)

# Get shape info from first EI
example_ei = units[unit_ids[0]]['ei']
num_channels, num_timepoints = example_ei.shape

# Allocate array
ax_ei_stack = np.zeros((num_units, num_channels, num_timepoints), dtype=np.float32)

# Fill it
for i, unit_id in enumerate(unit_ids):
    ax_ei_stack[i] = units[unit_id]['ei']




In [ ]:
target_units = [4, 54, 59, 9]

# target_units = [4, 54, 59]

window_start = 0#3003000
window_end = 10_000_000#3003000+3000
channel = 413

import matplotlib.pyplot as plt
# Collect spikes per unit
spikes_in_window = {}
for uid in target_units:
    spikes = units[uid]['spike_times']
    spikes_window = spikes[(spikes >= window_start) & (spikes < window_end)]
    spikes_in_window[uid] = spikes_window

# Extract raw data for the window
raw_snip = raw_data[window_start:window_end, channel].astype(np.float32)

# plt.figure(figsize=(12, 4))
# x = np.arange(window_start, window_end)
# plt.plot(x, raw_snip, color='black', linewidth=0.5, label='Raw data')

# # Colors for each unit
# colors = {4: 'red', 54: 'blue', 59: 'green', 9: 'magenta'}

# # Plot spike markers
# for uid in target_units:
#     spikes = spikes_in_window[uid]
#     for s in spikes:
#         plt.axvline(s, color=colors[uid], linestyle='--', alpha=0.7, label=f'Unit {uid}')
        
# # To avoid duplicate labels in legend
# handles, labels = plt.gca().get_legend_handles_labels()
# unique = dict(zip(labels, handles))
# plt.legend(unique.values(), unique.keys())

# plt.xlabel("Sample")
# plt.ylabel("ADC units")
# plt.title(f"Raw data on channel {channel} with spike times")
# plt.grid(True)
# plt.tight_layout()
# plt.show()


In [ ]:
C = 413
offsets = []

for uid in target_units:
    p2p = np.ptp(ax_ei_stack[uid], axis=1)  # [channels]
    main_ch = np.argmax(p2p)
    
    T_main = ax_ei_stack[uid, main_ch, :]
    idx_main = np.argmin(T_main) if np.min(T_main) < -np.max(T_main) else np.argmax(T_main)
    
    T_C = ax_ei_stack[uid, C, :]
    idx_c = np.argmin(T_C) if np.min(T_C) < -np.max(T_C) else np.argmax(T_C)
    
    offsets.append(idx_c - idx_main)

offsets_map = dict(zip(target_units, offsets))

print(offsets_map)

In [ ]:
templates_413 = {}

window_template = [-10,70]

for uid in target_units:
    T_C = ax_ei_stack[uid, C, :]  # template on 413
    p2p = np.ptp(ax_ei_stack[uid], axis=1)
    main_ch = np.argmax(p2p)
    
    T_main = ax_ei_stack[uid, main_ch, :]
    idx_main = np.argmin(T_main) if np.min(T_main) < -np.max(T_main) else np.argmax(T_main)

    center = idx_main + offsets_map[uid]

    start = center + window_template[0]
    end = center + window_template[1] + 1  # +1 because slicing is exclusive

    if start < 0 or end > T_C.shape[0]:
        print(f"Skipping unit {uid}: template cut out of bounds [{start}:{end}]")
        continue

    templates_413[uid] = T_C[start:end]

import matplotlib.pyplot as plt

plt.figure(figsize=(10, 3))

x = np.arange(window_template[0], window_template[1]+1)  # sample offsets relative to center

colors = {4: 'red', 54: 'blue', 59: 'green', 9: 'magenta'}

for uid in templates_413:
    T = templates_413[uid]
    plt.plot(x[:len(T)], T, label=f'Unit {uid}', color=colors[uid], alpha=0.8)

plt.xlabel("Samples relative to center")
plt.ylabel("ADC units")
plt.title(f"Templates on channel {C} (aligned)")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
chunk_len = window_end - window_start
generated = np.zeros(chunk_len, dtype=np.float32)

for uid in target_units:
    spikes = units[uid]['spike_times']
    template = templates_413[uid]
    offset = offsets_map[uid]
    
    # Filter spikes in chunk
    spikes_in_chunk = spikes[(spikes >= window_start) & (spikes < window_end)]
    
    for t_spike in spikes_in_chunk:
        arrival = t_spike + offset
        chunk_start_idx = int(arrival + window_template[0] - window_start)
        chunk_end_idx = chunk_start_idx + len(template)

        # Check bounds
        if chunk_start_idx < 0 or chunk_end_idx > chunk_len:
            print(f"Skipping spike at {t_spike} (would go out of bounds)")
            continue

        # Add template
        generated[chunk_start_idx:chunk_end_idx] += template

raw_snip = raw_data[window_start:window_end, C].astype(np.float32)

residual = raw_snip - generated

from scipy.signal import find_peaks

# Invert residual to find negative peaks as positive peaks
peaks, _ = find_peaks(-residual)

# Get peak values
neg_peak_values = residual[peaks]

# Keep only the negative ones (should be all negative by construction, but safe check)
neg_peak_values = neg_peak_values[neg_peak_values < 0]


import matplotlib.pyplot as plt

plt.figure(figsize=(8, 4))
plt.hist(neg_peak_values, bins=100, color='black', alpha=0.7)
plt.xlabel("Negative peak residual (ADC units)")
plt.ylabel("Count")
plt.title(f"Distribution of negative residual peaks (channel {C})")
plt.ylim(0, 25)
plt.grid(True)
plt.tight_layout()
plt.show()

# plt.figure(figsize=(8, 4))
# plt.hist(residual, bins=100, color='black', alpha=0.7)
# plt.xlabel("Residual (ADC units)")
# plt.ylabel("Count")
# plt.title(f"Residual distribution over {window_end - window_start} samples on channel {C}")
# plt.ylim(0, 200)  # set y-axis limit to 10k
# plt.grid(True)
# plt.tight_layout()
# plt.show()



In [ ]:
threshold = -100

# peaks: indices of peaks in residual (relative to chunk)
# residual[peaks]: their values

# Find peaks that are below threshold
mask = residual[peaks] < threshold
peak_indices_below_thresh = peaks[mask]

# Convert to original sample indices
sample_indices_below_thresh = peak_indices_below_thresh + window_start

print(f"Found {len(sample_indices_below_thresh)} negative peaks below {threshold}")


# for s_idx, val in zip(sample_indices_below_thresh, residual[peak_indices_below_thresh]):
#     print(f"Sample {s_idx}: residual peak {val}")


In [ ]:
import matplotlib.pyplot as plt

win_pre = 100
win_post = 100
snippet_len = win_pre + win_post + 1

plt.figure(figsize=(12, 4))

for i, s_idx in enumerate(sample_indices_below_thresh):
    start = s_idx - win_pre
    end = s_idx + win_post + 1  # +1 because slice end is exclusive

    # Check bounds
    if start < 0 or end > raw_data.shape[0]:
        print(f"Skipping out-of-bounds snippet at sample {s_idx}")
        continue

    snippet = raw_data[start:end, C].astype(np.float32)
    x = np.arange(-win_pre, win_post + 1)

    plt.plot(x, snippet, alpha=0.7, label=f"Peak at {s_idx}")

plt.xlabel("Samples relative to peak")
plt.ylabel("ADC units")
plt.title(f"Raw data snippets around negative residual peaks (channel {C})")
plt.grid(True)

# Optional: limit legend to avoid clutter if many peaks
handles, labels = plt.gca().get_legend_handles_labels()
if len(handles) > 10:
    plt.legend(handles[:10], labels[:10], title="First 10 peaks")
else:
    plt.legend()

plt.tight_layout()
plt.show()

plt.figure(figsize=(12, 4))

for s_idx in sample_indices_below_thresh:
    start = s_idx - win_pre
    end = s_idx + win_post + 1

    # Check bounds
    if start < 0 or end > len(residual):
        print(f"Skipping out-of-bounds residual snippet at {s_idx}")
        continue

    snippet = residual[start:end]
    x = np.arange(-win_pre, win_post + 1)

    plt.plot(x, snippet, alpha=0.7)

plt.xlabel("Samples relative to residual peak")
plt.ylabel("Residual (ADC units)")
plt.title(f"Residual snippets centered on negative residual peaks (channel {C})")
plt.grid(True)
plt.tight_layout()
plt.show()



In [ ]:
from sklearn.decomposition import PCA

win_pre_pca = 25
win_post_pca = 25
snip_len_pca = win_pre_pca + win_post_pca + 1

snippets = []

for s_idx in sample_indices_below_thresh:
    start = s_idx - win_pre_pca
    end = s_idx + win_post_pca + 1

    if start < 0 or end > len(residual):
        print(f"Skipping out-of-bounds residual snippet at {s_idx}")
        continue

    snippet = residual[start:end]
    snippets.append(snippet)

snippets = np.array(snippets)  # shape: [n_snippets, 51]

pca = PCA(n_components=2)
pcs = pca.fit_transform(snippets)

plt.figure(figsize=(6, 6))
plt.scatter(pcs[:, 0], pcs[:, 1], alpha=0.7, color='black')
plt.xlabel('PC1')
plt.ylabel('PC2')
plt.title('PCA of residual snippets around negative peaks')
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
from verify_cluster import verify_cluster

# parameters for verify_cluster
params = {
    'window': (-20, 60),
    'min_spikes': 100,
    'ei_sim_threshold': 0.75,
    'k_start': 4,
    'k_refine': 2
}

# choose cell ID (this is index of the cell, not vision ID)
spike_times = sample_indices_below_thresh

# run recursive clustering
clusters = verify_cluster(
    spike_times=spike_times,
    dat_path=dat_path,
    params=params
)

print(f"Returned {len(clusters)} clean subclusters")
for i, cl in enumerate(clusters):
    print(f"  Cluster {i}: {len(cl['inds'])} spikes")

print("Success")

In [ ]:
import analyze_clusters
import importlib
importlib.reload(analyze_clusters)

# plot EI (Vision style), ISI, firing rate, time course (from one pixel) and STA (single frame with strongest pixel)
analyze_clusters.analyze_clusters(clusters,
                 spike_times=spike_times,
                 sampling_rate=20000,
                 dat_path='/Volumes/Lab/Users/alexth/axolotl/201703151_data001.dat',
                 h5_path='/Volumes/Lab/Users/alexth/axolotl/201703151_kilosort_data001_spike_times.h5',
                 triggers_mat_path='/Volumes/Lab/Users/alexth/axolotl/trigger_in_samples_201703151.mat',
                 cluster_ids=None,
                 lut=None,
                 sta_depth=30,
                 sta_offset=0,
                 sta_chunk_size=1000,
                 sta_refresh=2,
                 ei_scale=3,
                 ei_cutoff=0.08)

In [ ]:
import plot_ei_waveforms
import importlib
importlib.reload(plot_ei_waveforms)

snips_full, valid_spike_times = axolotl_utils_ram.extract_snippets_fast_ram(
    raw_data=raw_data,
    spike_times=sample_indices_below_thresh,
    selected_channels=np.arange(n_channels),
    window=window,
)

final_ei = np.mean(snips_full, axis=2)


plt.figure(figsize=(10,10))
plot_ei_waveforms.plot_ei_waveforms(final_ei, ei_positions, scale=70.0, box_height=1.0, box_width=50.0, colors='black')
plt.show()


In [ ]:

sim = compare_eis(ax_ei_stack, final_ei, 10).reshape(-1) 
best_idx = np.argmax(sim)
best_sim = sim[best_idx]

print(f"Closest Axolotl unit {best_idx} with similarity {best_sim:.3f}")

In [ ]:
import axolotl_utils_ram
import importlib
importlib.reload(axolotl_utils_ram)

from scipy.io import loadmat

triggers_mat_path='/Volumes/Lab/Users/alexth/axolotl/trigger_in_samples_201703151.mat'
triggers_sec = loadmat(triggers_mat_path)['triggers'].flatten()

axolotl_utils_ram.analyze_single_unit(spike_samples = sample_indices_below_thresh,
                        sampling_rate = 20000,
                        triggers_sec = triggers_sec,
                        ei = final_ei,
                        ei_positions = ei_positions,
                        lut=None,
                        sta_depth=20,
                        sta_offset=0,
                        sta_chunk_size=1000,
                        sta_refresh=2,
                        ei_scale=3,
                        ei_cutoff=0.05,
                        isi_max_ms=200,
                        sigma_ms=2500,
                        dt_ms=1000)

In [ ]:
explained_peaks = []

for peak_sample in sample_indices_below_thresh:
    found = False
    for uid in target_units:
        spikes = units[uid]['spike_times']
        offset = offsets_map[uid]
        arrivals = spikes + offset
        
        # Check if any arrival is within ±1 of peak_sample
        if np.any(np.abs(arrivals - peak_sample) <= 2):
            found = True
            print(f"Residual peak at {peak_sample} explained by unit {uid}")
            break
    
    if not found:
        print(f"Residual peak at {peak_sample} NOT explained by any target unit")
    
    explained_peaks.append(found)


In [ ]:
chunk_len = window_end - window_start
generated = np.zeros(chunk_len, dtype=np.float32)

for uid in target_units:
    spikes = units[uid]['spike_times']
    template = templates_413[uid]
    offset = offsets_map[uid]
    
    # Filter spikes in chunk
    spikes_in_chunk = spikes[(spikes >= window_start) & (spikes < window_end)]
    
    for t_spike in spikes_in_chunk:
        arrival = t_spike + offset
        chunk_start_idx = int(arrival + window_template[0] - window_start)
        chunk_end_idx = chunk_start_idx + len(template)

        # Check bounds
        if chunk_start_idx < 0 or chunk_end_idx > chunk_len:
            print(f"Skipping spike at {t_spike} (would go out of bounds)")
            continue

        # Add template
        generated[chunk_start_idx:chunk_end_idx] += template

raw_snip = raw_data[window_start:window_end, C].astype(np.float32)
x = np.arange(window_start, window_end)


residual = raw_snip - generated

import matplotlib.pyplot as plt

plt.figure(figsize=(12, 4))
plt.plot(x, raw_snip, label='Original raw data', color='black', linewidth=0.5)
plt.plot(x, residual, label='Original raw data', color='green', linewidth=0.5)
plt.plot(x, generated, label='Generated from templates', color='red', linewidth=0.7)

plt.xlabel("Sample")
plt.ylabel("ADC units")
plt.title(f"Original vs generated signal on channel {C}")
plt.legend()
plt.xlim(2462-100, 2462+100)
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
residual = raw_snip - generated
plt.figure(figsize=(12, 4))
x = np.arange(window_start, window_end)

plt.plot(x, residual, label='Residual (raw - generated)', color='black', linewidth=0.7)

# Colors for units
colors = {4: 'red', 54: 'blue', 59: 'green', 9: 'magenta'}

# Mark spikes on residual
for uid in target_units:
    spikes = units[uid]['spike_times']
    spikes_in_chunk = spikes[(spikes >= window_start) & (spikes < window_end)]
    for s in spikes_in_chunk:
        plt.axvline(s, color=colors[uid], linestyle='--', alpha=0.5, label=f'Unit {uid}')

# Remove duplicate labels
handles, labels = plt.gca().get_legend_handles_labels()
unique = dict(zip(labels, handles))
plt.legend(unique.values(), unique.keys())

plt.xlabel("Sample")
plt.ylabel("Residual (ADC units)")
plt.title(f"Residuals on channel {C} with spike locations marked")
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
window_size = 5
# Use 'valid' to avoid padding artifacts, or 'same' to keep same length
residual_ma = np.convolve(residual, np.ones(window_size)/window_size, mode='same')

plt.figure(figsize=(12, 4))
x = np.arange(window_start, window_end)

plt.plot(x, residual_ma, label=f'Residual moving avg (win={window_size})', color='black', linewidth=1.0)

# Mark spikes
for uid in target_units:
    spikes = units[uid]['spike_times']
    spikes_in_chunk = spikes[(spikes >= window_start) & (spikes < window_end)]
    for s in spikes_in_chunk:
        plt.axvline(s, color=colors[uid], linestyle='--', alpha=0.5, label=f'Unit {uid}')

# Remove duplicate labels
handles, labels = plt.gca().get_legend_handles_labels()
unique = dict(zip(labels, handles))
plt.legend(unique.values(), unique.keys())

plt.xlabel("Sample")
plt.ylabel("Residual (ADC units)")
plt.title(f"Residual + moving avg on channel {C}")
plt.grid(True)
plt.tight_layout()
plt.show()


### Collision model

In [ ]:
ax_ei_stack = np.stack(ax_ei_list, axis=0)

print(ax_ei_stack.shape)

In [ ]:
import axolotl_utils_ram
import importlib
importlib.reload(axolotl_utils_ram)


units = axolotl_utils_ram.load_units_from_h5(h5_out_path)

# Get all unit IDs (sorted, to ensure order is clear)
unit_ids = sorted(units.keys())
num_units = len(unit_ids)

# Get shape info from first EI
example_ei = units[unit_ids[0]]['ei']
num_channels, num_timepoints = example_ei.shape

# Allocate array
ax_ei_stack = np.zeros((num_units, num_channels, num_timepoints), dtype=np.float32)

# Fill it
for i, unit_id in enumerate(unit_ids):
    ax_ei_stack[i] = units[unit_id]['ei']


p2p = ax_ei_stack.max(axis=2) - ax_ei_stack.min(axis=2)  # [93, 512]

pairs = axolotl_utils_ram.candidate_pairs_simple(p2p, thr=0.1)

# print(pairs)

# Find max unit ID (in case unit IDs are sparse or out of order)
max_unit_id = max(units.keys())

# Build list where spike_times_list[unit_id] = spike_times
# Use None or empty array for missing units if any
spike_times_list = [None] * (max_unit_id + 1)

for unit_id, data in units.items():
    spike_times_list[unit_id] = data['spike_times']


In [ ]:
cell_id = 2

A, B, shared = pairs[cell_id]
C = shared[5]

tA = spike_times_list[A]
tB = spike_times_list[B]

T_A_C = ax_ei_stack[A, C, :]
T_B_C = ax_ei_stack[B, C, :]

pkA = np.argmin(T_A_C) if np.min(T_A_C) < -np.max(T_A_C) else np.argmax(T_A_C)
pkB = np.argmin(T_B_C) if np.min(T_B_C) < -np.max(T_B_C) else np.argmax(T_B_C)

hitsA, hitsB = axolotl_utils_ram.find_collisions_1ch(tA, tB, pkA, pkB, delta=15)

print(len(hitsA))
# print("Collision spike indices in A:", hitsA)
# print("Collision spike indices in B:", hitsB)

In [ ]:
soloA, soloB = axolotl_utils_ram.find_solo_1ch(tA, tB, pkA, pkB, delta=60)

In [ ]:
p2p_A = np.ptp(ax_ei_stack[A], axis=1)  # [channels]
main_ch_A = np.argmax(p2p_A)
T_A_main = ax_ei_stack[A, main_ch_A, :]
idx_main_A = np.argmin(T_A_main) if np.min(T_A_main) < -np.max(T_A_main) else np.argmax(T_A_main)
T_A_C = ax_ei_stack[A, C, :]
idx_c_A = np.argmin(T_A_C) if np.min(T_A_C) < -np.max(T_A_C) else np.argmax(T_A_C)
offset_A = idx_c_A - idx_main_A

print(offset_A)

p2p_B = np.ptp(ax_ei_stack[B], axis=1)  # [channels]
main_ch_B = np.argmax(p2p_B)
T_B_main = ax_ei_stack[B, main_ch_B, :]
idx_main_B = np.argmin(T_B_main) if np.min(T_B_main) < -np.max(T_B_main) else np.argmax(T_B_main)
T_B_C = ax_ei_stack[B, C, :]
idx_c_B = np.argmin(T_B_C) if np.min(T_B_C) < -np.max(T_B_C) else np.argmax(T_B_C)
offset_B = idx_c_B - idx_main_B

print(offset_B)

win_pre = 40
win_post = 40
win_len = win_pre + win_post  # 80

raw_snippets = []  # for storing y
template_As = []   # for storing aligned A templates
template_Bs = []   # for storing aligned B templates
shifts = []        # for storing shift values

# For A
snipsA, _ = axolotl_utils_ram.extract_snippets_fast_ram(
    raw_data=raw_data,
    spike_times=tA + offset_A,
    selected_channels=np.array([C]),
    window=(-win_pre, win_post - 1),  # 80 samples
)
template_A = snipsA[0].mean(axis=1)  # [80]

# For B — larger window for shifting
snipsB, _ = axolotl_utils_ram.extract_snippets_fast_ram(
    raw_data=raw_data,
    spike_times=tB + offset_B,
    selected_channels=np.array([C]),
    window=(-win_pre - 30, win_post + 30 - 1),  # 80 + 60 = 140 samples
)
template_B_full = snipsB[0].mean(axis=1)  # [140]



In [ ]:

import matplotlib.pyplot as plt

# Example shift value: how much B's peak should move to align with A's
# For example: let's align B's peak at sample 40
desired_shift = 0  # no shift → peak at 40
# desired_shift = 10  # shift B's peak to 40 + 10 = 50
# desired_shift = -10  # shift B's peak to 40 - 10 = 30

# Determine cut indices
start_B = 30 - desired_shift
end_B = start_B + 80

if start_B < 0 or end_B > len(template_B_full):
    raise ValueError(f"Shift {desired_shift} causes out-of-bounds slice: [{start_B}:{end_B}]")

TB = template_B_full[start_B:end_B]

x = np.arange(80)

plt.figure(figsize=(8, 3))
plt.plot(x, template_A, label='Template A (centered)', color='blue')
plt.plot(x, TB, label=f'Template B (shifted by {desired_shift} samples)', color='red')
plt.xlabel("Sample")
plt.ylabel("ADC units")
plt.title("Average templates overlay (aligned peak position)")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:


for k in range(len(hitsA)):
    # 1️⃣ Spike times
    tA_hit = tA[hitsA[k]]
    tB_hit = tB[hitsB[k]]

    # 2️⃣ Compute arrival times at C
    centerA = tA_hit + offset_A
    centerB = tB_hit + offset_B

    # 3️⃣ Compute shift
    shift = centerB - centerA
    shifts.append(shift)

    # 4️⃣ Extract raw data snippet centered on centerA
    start = centerA - win_pre
    end = centerA + win_post

    if start < 0 or end >= raw_data.shape[0]:
        print(f"Skipping out-of-bounds collision at k={k}")
        continue

    y = raw_data[start:end, C].astype(np.float32)
    raw_snippets.append(y)

    # 5️⃣ Build template A (centered)
    # A's template already has peak at 40
    template_As.append(template_A)  # template_A = your 80-sample mean template for A

    # 6️⃣ Build template B (shifted)
    shift_int = int(round(shift))
    start_B = 30 - shift_int
    end_B = start_B + win_len

    if start_B < 0 or end_B > template_B_full.shape[0]:
        print(f"Skipping B template out-of-bounds at k={k}")
        continue

    TB_aligned = template_B_full[start_B:end_B]
    template_Bs.append(TB_aligned)

In [ ]:
import matplotlib.pyplot as plt

k_plot = 3  # index of the collision pair you want to plot

TA = template_As[k_plot]
TB = template_Bs[k_plot]

x = np.arange(len(TA))  # sample positions (0..79)

plt.figure(figsize=(8, 3))
plt.plot(x, TA, label='Template A (centered)', color='red')
plt.plot(x, TB, label='Template B (shifted)', color='blue')
plt.plot(x, raw_snippets[k_plot], label='raw', color='black')
plt.xlabel("Sample")
plt.ylabel("ADC units")
plt.title(f"Collision template overlay at k={k_plot}")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
import axolotl_utils_ram
import importlib
importlib.reload(axolotl_utils_ram)

alphas = []
betas = []
residuals = []

for y, TA, TB in zip(raw_snippets, template_As, template_Bs):
    alpha, beta, rnorm = axolotl_utils_ram.fit_two_templates_bounded(y, TA, TB, lower=0.75, upper=1.25)
    alphas.append(alpha)
    betas.append(beta)
    residuals.append(rnorm)

In [ ]:
spike_idx = 39

y = raw_snippets[spike_idx]
TA = template_As[spike_idx]
TB = template_Bs[spike_idx]
alpha = alphas[spike_idx]
beta = betas[spike_idx]
rnorm = residuals[spike_idx]

fit = alpha * TA + beta * TB
residual = y - fit

x = np.arange(len(y))

import matplotlib.pyplot as plt

plt.figure(figsize=(10, 3))
plt.plot(x, y, color='green', label='Raw data')
plt.plot(x, alpha * TA, color='red', label=f'Template A (α={alpha:.2f})')
plt.plot(x, beta * TB, color='blue', label=f'Template B (β={beta:.2f})')
plt.plot(x, residual, color='black', label=f'Residual (norm={rnorm:.2f})')

plt.title(f'Collision fit at spike index {spike_idx}')
plt.xlabel("Sample")
plt.ylabel("ADC units")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(15, 4))

# Alpha histogram
plt.subplot(1, 4, 1)
plt.hist(alphas, bins=50, color='red', alpha=0.7)
plt.xlabel('Alpha (A amplitude)')
plt.ylabel('Count')
plt.title('Distribution of alpha')

# Beta histogram
plt.subplot(1, 4, 2)
plt.hist(betas, bins=50, color='blue', alpha=0.7)
plt.xlabel('Beta (B amplitude)')
plt.title('Distribution of beta')

# Residual norm histogram
plt.subplot(1, 4, 3)
plt.hist(residuals, bins=50, color='black', alpha=0.7)
plt.title(f"Residuals {len(residuals)}")



# Convert to arrays if they aren't already
alphas = np.array(alphas)
betas = np.array(betas)
residuals = np.array(residuals)

# Identify fits where alpha and beta are not at bounds
bound_tol = 1e-6  # tolerance to consider "at bound"
alpha_not_bound = (alphas > (0.75 + bound_tol)) & (alphas < (1.25 - bound_tol))
beta_not_bound = (betas > (0.75 + bound_tol)) & (betas < (1.25 - bound_tol))

# Combined mask
valid_mask = alpha_not_bound & beta_not_bound

# Extract residuals for these cases
residuals_valid = residuals[valid_mask]

# Plot
plt.subplot(1, 4, 4)
plt.hist(residuals_valid, bins=50, color='black', alpha=0.7)
plt.xlabel('Residual norm')
plt.ylabel('Count')
plt.title(f"Residuals {len(residuals_valid)}")
plt.grid(True)
plt.tight_layout()
plt.show()

plt.tight_layout()
plt.show()


In [ ]:
import axolotl_utils_ram
import importlib
importlib.reload(axolotl_utils_ram)


for k in range(len(soloA)):
    # 1️⃣ Spike times
    tA_hit = tA[soloA[k]]

    # 2️⃣ Compute arrival times at C
    centerA = tA_hit + offset_A

    # 4️⃣ Extract raw data snippet centered on centerA
    start = centerA - win_pre
    end = centerA + win_post

    if start < 0 or end >= raw_data.shape[0]:
        print(f"Skipping out-of-bounds collision at k={k}")
        continue

    y = raw_data[start:end, C].astype(np.float32)
    raw_snippets.append(y)

    # 5️⃣ Build template A (centered)
    # A's template already has peak at 40
    template_As.append(template_A)  # template_A = your 80-sample mean template for A


template_Bs = [np.zeros_like(template_A) for _ in range(len(template_As))]

alphas = []
betas = []
residuals = []

for y, TA, TB in zip(raw_snippets, template_As, template_Bs):
    alpha, beta, rnorm = axolotl_utils_ram.fit_two_templates_bounded(y, TA, TB, lower=0.75, upper=1.25)
    alphas.append(alpha)
    betas.append(beta)
    residuals.append(rnorm)

In [ ]:
spike_idx = 12

y = raw_snippets[spike_idx]
TA = template_As[spike_idx]
TB = template_Bs[spike_idx]
alpha = alphas[spike_idx]
beta = betas[spike_idx]
rnorm = residuals[spike_idx]

fit = alpha * TA + beta * TB
residual = y - fit

x = np.arange(len(y))

import matplotlib.pyplot as plt

plt.figure(figsize=(10, 3))
plt.plot(x, y, color='green', label='Raw data')
plt.plot(x, alpha * TA, color='red', label=f'Template A (α={alpha:.2f})')
plt.plot(x, beta * TB, color='blue', label=f'Template B (β={beta:.2f})')
plt.plot(x, residual, color='black', label=f'Residual (norm={rnorm:.2f})')

plt.title(f'Collision fit at spike index {spike_idx}')
plt.xlabel("Sample")
plt.ylabel("ADC units")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
import axolotl_utils_ram
import importlib
importlib.reload(axolotl_utils_ram)

template_B = template_B_full[30:110]

raw_snippets = []
template_Bs = []
template_As = []

for k in range(len(soloB)):
    tB_hit = tB[soloB[k]]
    centerB = tB_hit + offset_B

    start = centerB - win_pre
    end = centerB + win_post

    if start < 0 or end >= raw_data.shape[0]:
        print(f"Skipping out-of-bounds solo B at k={k}")
        continue

    y = raw_data[start:end, C].astype(np.float32)
    raw_snippets.append(y)

    template_Bs.append(template_B)  # your 80-sample mean template for B
    template_As.append(np.zeros_like(template_B))  # zero A template

alphas = []
betas = []
residuals = []

for y, TA, TB in zip(raw_snippets, template_As, template_Bs):
    alpha, beta, rnorm = axolotl_utils_ram.fit_two_templates_bounded(y, TA, TB, lower=0.75, upper=1.25)
    alphas.append(alpha)
    betas.append(beta)
    residuals.append(rnorm)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(15, 4))

# Alpha histogram
plt.subplot(1, 4, 1)
plt.hist(alphas, bins=50, color='red', alpha=0.7)
plt.xlabel('Alpha (A amplitude)')
plt.ylabel('Count')
plt.title('Distribution of alpha')

# Beta histogram
plt.subplot(1, 4, 2)
plt.hist(betas, bins=50, color='blue', alpha=0.7)
plt.xlabel('Beta (B amplitude)')
plt.title('Distribution of beta')

# Residual norm histogram
plt.subplot(1, 4, 3)
plt.hist(residuals, bins=50, color='black', alpha=0.7)
plt.title(f"Residuals {len(residuals)}")



# Convert to arrays if they aren't already
alphas = np.array(alphas)
betas = np.array(betas)
residuals = np.array(residuals)

# Identify fits where alpha and beta are not at bounds
bound_tol = 1e-6  # tolerance to consider "at bound"
alpha_not_bound = (alphas > (0.75 + bound_tol)) & (alphas < (1.25 - bound_tol))
beta_not_bound = (betas > (0.75 + bound_tol)) & (betas < (1.25 - bound_tol))

# Combined mask
valid_mask = alpha_not_bound & beta_not_bound

# Extract residuals for these cases
residuals_valid = residuals[valid_mask]

# Plot
plt.subplot(1, 4, 4)
plt.hist(residuals_valid, bins=50, color='black', alpha=0.7)
plt.xlabel('Residual norm')
plt.ylabel('Count')
plt.title(f"Residuals {len(residuals_valid)}")
plt.grid(True)
plt.tight_layout()
plt.show()

plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt

win_pre = 50
win_post = 50
win_len = win_pre + win_post + 1

# Pick a few hitsA indices to inspect (e.g. first 5)
for idx in hitsA[:5]:
    center = tA[idx]
    
    start = center - win_pre
    end = center + win_post + 1  # +1 because Python slicing is exclusive
    
    if start < 0 or end > raw_data.shape[0]:
        print(f"Skipping spike at {center} (out of bounds)")
        continue
    
    trace = raw_data[start:end, C]
    
    plt.figure(figsize=(8, 2))
    plt.plot(np.arange(-win_pre, win_post + 1), trace, color='black')
    plt.title(f"Unit A spike at {center} on channel {C}")
    plt.xlabel("Samples relative to center")
    plt.ylabel("ADC units")
    plt.grid(True)
    plt.tight_layout()
    plt.show()


### Random debug stuff

In [ ]:
import matplotlib.pyplot as plt

from scipy.stats import chi2
thr_vis  = chi2.ppf(0.9,  df=2)     # illustration
thr_cut  = chi2.ppf(0.9, df=2)     # discard

final_outlier_inds      = np.where(d2 > thr_vis)[0]
final_outlier_inds_max  = np.where(d2 > thr_cut)[0]




print(f"LOF rejects: {len(lof_inds)}")

final_outlier_inds      = lof_inds


# Assume labels_kmeans contains the 0/1 cluster assignments from k-means on best_proj
# And pcs is your [N, 2] PC1/PC2 array (z-scored)
pcs = pcs_z
plt.figure(figsize=(4, 4))
plt.scatter(pcs[:, 0], pcs[:, 1], s=5, alpha=0.7)
plt.scatter(pcs[final_outlier_inds_max, 0], pcs[final_outlier_inds_max, 1], s=15, color="green",alpha=1)
plt.scatter(pcs[final_outlier_inds, 0], pcs[final_outlier_inds, 1], s=3, color="red",alpha=1)
plt.xlabel("PC1 (z-score)")
plt.ylabel("PC2 (z-score)")
plt.title(f"PC1 vs PC2 scatter\Hartigan p = {p:.3f}")
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(6, 5))
plt.scatter(pc1_z, pc2_z, s=2, alpha=0.5)
plt.title("PC1 z-scores for Ref Channel Waveforms")
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.grid(True)
plt.tight_layout()
plt.show()

thresh_pc = 6
final_outlier_mask_max = (np.abs(pc1_z) > thresh_pc) | (np.abs(pc2_z) > thresh_pc) # or -2.5, tune this
final_outlier_inds_max = np.where(final_outlier_mask_max)[0]

print(len(final_outlier_inds_max))
# print(final_outlier_inds_max)

if final_outlier_inds_max.size > 0:
    from scipy.spatial.distance import cdist

    # Combine PC1 and PC2 z-scores
    pcs_all = np.column_stack((pc1_z, pc2_z))

    # Split into bad and good
    bad_pcs = pcs_all[final_outlier_inds_max]
    good_mask = np.ones(len(pcs_all), dtype=bool)
    good_mask[final_outlier_inds_max] = False
    good_pcs = pcs_all[good_mask]

    N = min(20, len(good_pcs))

    bad_good_means = []
    good_good_means = []
    ratios = []

    for bad in bad_pcs:
        # Distance to all good spikes
        dists = np.linalg.norm(good_pcs - bad, axis=1)
        nearest_inds = np.argsort(dists)[:N]
        nearest_good = good_pcs[nearest_inds]

        # Mean distance bad -> nearest N good
        mean_bad_good = np.mean(dists[nearest_inds])
        bad_good_means.append(mean_bad_good)

        # Mean pairwise distance among those N good spikes
        gg_dists = cdist(nearest_good, nearest_good)
        mean_good_good = np.mean(gg_dists[np.triu_indices_from(gg_dists, k=1)])
        good_good_means.append(mean_good_good)

        # Ratio
        ratios.append(mean_bad_good / (mean_good_good + 1e-8))

    bad_good_means = np.array(bad_good_means)
    good_good_means = np.array(good_good_means)
    ratios = np.array(ratios)
    # Summary
    print(f"Mean bad-good distance: {np.mean(bad_good_means):.4f}")
    print(f"Mean good-good distance: {np.mean(good_good_means):.4f}")
    print(f"Mean ratio (bad-good / good-good): {np.mean(ratios):.4f}")

    # Optional: print distribution
    # for i, (bg, gg, r) in enumerate(zip(bad_good_means, good_good_means, ratios)):
    #     print(f"Bad spike {i}: bad-good = {bg:.4f}, good-good = {gg:.4f}, ratio = {r:.2f}")
    # final_outlier_inds_max = final_outlier_inds_max[ratios>3.5]

N = snips_baselined.shape[2] # or however you define total spike count
labels = np.zeros(N, dtype=int)
thresh_pc = 2
final_outlier_mask = (np.abs(pc1_z) > thresh_pc) | (np.abs(pc2_z) > thresh_pc) # or -2.5, tune this
final_outlier_inds = np.where(final_outlier_mask)[0]

labels[final_outlier_inds] = 1

thresh_pc = 6
final_outlier_mask_max = (np.abs(pc1_z) > thresh_pc) | (np.abs(pc2_z) > thresh_pc) # or -2.5, tune this
final_outlier_inds_max = np.where(final_outlier_mask_max)[0]

labels[final_outlier_inds_max] = 2

merged_clusters, sim, n_bad_channels = axolotl_utils_ram.merge_similar_clusters(snips_baselined, labels, max_lag=3, p2p_thresh=30.0, amp_thresh=-20, cos_thresh=0.9)

print(sim)
print(n_bad_channels)
print(N)
print(len(merged_clusters))
for i, arr in enumerate(merged_clusters):
    print(f"Array {i} length: {len(arr)}")



In [ ]:
from verify_cluster import verify_cluster

# parameters for verify_cluster
params = {
    'window': (-20, 60),
    'min_spikes': 100,
    'ei_sim_threshold': 0.75,
    'k_start': 10,
    'k_refine': 4
}

# choose cell ID (this is index of the cell, not vision ID)
spike_times = spikes

# run recursive clustering
clusters = verify_cluster(
    spike_times=spike_times,
    dat_path=dat_path,
    params=params
)

print(f"Returned {len(clusters)} clean subclusters")
for i, cl in enumerate(clusters):
    print(f"  Cluster {i}: {len(cl['inds'])} spikes")

print("Success")

In [ ]:
import analyze_clusters
import importlib
importlib.reload(analyze_clusters)

# plot EI (Vision style), ISI, firing rate, time course (from one pixel) and STA (single frame with strongest pixel)
analyze_clusters.analyze_clusters(clusters,
                 spike_times=spike_times,
                 sampling_rate=20000,
                 dat_path='/Volumes/Lab/Users/alexth/axolotl/201703151_data001.dat',
                 h5_path='/Volumes/Lab/Users/alexth/axolotl/201703151_kilosort_data001_spike_times.h5',
                 triggers_mat_path='/Volumes/Lab/Users/alexth/axolotl/trigger_in_samples_201703151.mat',
                 cluster_ids=None,
                 lut=None,
                 sta_depth=30,
                 sta_offset=0,
                 sta_chunk_size=1000,
                 sta_refresh=2,
                 ei_scale=3,
                 ei_cutoff=0.08)

### END of main pipeline

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from plot_ei_waveforms import plot_ei_waveforms

# --- PC1 vs PC2 scatter plot ---
plt.figure(figsize=(6, 5))
unique_labels = np.unique(labels_pre)
print(unique_labels)
colors = plt.cm.tab10.colors  # or any colormap you like

for i, label in enumerate(unique_labels):
    mask = labels_pre == label
    color = colors[i % len(colors)]
    plt.scatter(pcs_pre[mask, 0], pcs_pre[mask, 1], s=10, color=color, alpha=0.7, label=f"Cluster {label}")

plt.xlabel("PC1")
plt.ylabel("PC2")
plt.title("Cluster PCA Scatter")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

# --- Plot EIs for each cluster ---
for i, ei in enumerate(cluster_eis_pre):
    plt.figure(figsize=(15,5))
    plot_ei_waveforms(
        ei=ei,                 # list of EIs
        positions=ei_positions,
        ref_channel=ref_channel,
        scale=70.0,
        box_height=1.5,
        box_width=50,
        linewidth=0.5,
        alpha=0.9,
        colors='black'
    )


In [ ]:
sampling_rate = 20000

# --- Load spike times and electrode positions from HDF5 ---
all_spikes = {}
with h5py.File(h5_in_path, 'r') as f:
    unit_ids = sorted(f['/spikes'].keys(), key=lambda x: int(x.split('_')[1]))
    for uid in unit_ids:
        unit_index = int(uid.split('_')[1])
        raw = f[f'/spikes/{uid}'][:]
        if raw.ndim == 1 and raw.shape[0] == 1:
            spikes_sec = np.array(raw[0]).flatten()
        else:
            spikes_sec = np.array(raw).flatten()
        spikes_samples = np.round(spikes_sec * sampling_rate).astype(np.int32)
        all_spikes[unit_index] = spikes_samples


In [ ]:
ks_spikes = all_spikes[56]
print(len(ks_spikes))

import numpy as np
import matplotlib.pyplot as plt


ref_channel = 51

# Define window around each spike
pre, post = 20, 60
snip_len = pre + post + 1

# Allocate array to hold all snippets
snippets = []

for s in ss:#ks_spikes[:1000]:
    if s - pre >= 0 and s + post < raw_data.shape[0]:
        snippet = raw_data[s - pre : s + post + 1, ref_channel]
        snippets.append(snippet)

snippets = np.array(snippets)  # shape: [n_spikes, snip_len]

# Plot all snippets
plt.figure(figsize=(10, 9))
for i in range(snippets.shape[0]):
    plt.plot(snippets[i], color='black', alpha=0.1, linewidth=0.5)

plt.title(f"Ref channel {ref_channel} waveforms at {len(snippets)} spikes")
plt.xlabel("Sample index (relative to spike)")
plt.ylabel("Amplitude")
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
ks_spikes = all_spikes[56]

import numpy as np
import matplotlib.pyplot as plt


ref_channel = 51
amps = raw_data[ks_spikes, ref_channel]
plt.hist(amps, bins=100)
plt.axvline(-50, color='red', linestyle='--')



In [ ]:
ks_spikes = all_spikes[56]
# ks_spikes = ks_spikes[amps > -50]

print(len(ks_spikes))

ref_channel = 51
fs = 20000  # sampling rate in Hz
segment_len = fs  # 1 second = 20,000 samples

trace = raw_data[:, ref_channel].astype(np.float32)
n_samples = len(trace)

# Create baseline-corrected version of the trace
trace_corrected = np.empty_like(trace)

n_segments = (n_samples + segment_len - 1) // segment_len  # ceil division
for i in range(n_segments):
    start = i * segment_len
    end = min(start + segment_len, n_samples)
    segment = trace[start:end]
    trace_corrected[start:end] = segment - np.mean(segment)

# Now extract snippets from corrected trace
pre, post = 20, 60
snip_len = pre + post + 1
snippets = []

for s in ks_spikes[:1000]:
    if s - pre >= 0 and s + post < trace_corrected.shape[0]:
        snippet = trace_corrected[s - pre : s + post + 1]
        snippets.append(snippet)

snippets = np.array(snippets)  # shape: [n_spikes, snip_len]

# Plot
plt.figure(figsize=(10, 9))
for i in range(snippets.shape[0]):
    plt.plot(snippets[i], color='black', alpha=0.1, linewidth=0.5)

plt.title(f"Ref channel {ref_channel} waveforms at {len(snippets)} spikes\n(1s segment-wise baseline subtraction)")
plt.xlabel("Sample index (relative to spike)")
plt.ylabel("Amplitude")
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
amps = trace_corrected[ks_spikes]
plt.hist(amps, bins=100)
plt.axvline(-50, color='red', linestyle='--')


In [ ]:
import numpy as np
import h5py

unit_id_1 = 1011
max_diff = 10 # in samples

# --- Load spike times ---
with h5py.File(h5_out_path, 'r') as h5:
    spikes_1 = h5[f'unit_{unit_id_1}']['spike_times'][:]


spikes_2 = ks_spikes
# --- Sort spike times (for efficiency) ---
spikes_1 = np.sort(spikes_1)
spikes_2 = np.sort(spikes_2)

# --- Match spikes within max_diff ---
matches = []
j_start = 0

for i, t1 in enumerate(spikes_1):
    while j_start < len(spikes_2) and spikes_2[j_start] < t1 - max_diff:
        j_start += 1

    j = j_start
    while j < len(spikes_2) and spikes_2[j] <= t1 + max_diff:
        if abs(t1 - spikes_2[j]) <= max_diff:
            matches.append((i, j))
        j += 1

# --- Output ---
print(f"Found {len(matches)} matched spikes between unit {unit_id_1} and ks")
for i, j in matches:
    print(f"Spike1: {spikes_1[i]}, Spike2: {spikes_2[j]}")

# --- Build set of matched indices in spikes_2 ---
matched_j = set(j for _, j in matches)

# --- Extract unmatched spike times from spikes_2 ---
unmatched_spikes_2 = [spikes_2[j] for j in range(len(spikes_2)) if j not in matched_j]

print(f"Found {len(unmatched_spikes_2)} unmatched spikes in unit ks")
unmatched_spikes_2 = np.array(unmatched_spikes_2)


### Read from h5 results

In [ ]:
import h5py

unit_id = 40 # or whatever unit you're checking

with h5py.File(h5_out_path, 'r') as h5:
    group = h5[f'unit_{unit_id}']
    
    spike_times = group['spike_times'][:]
    ei = group['ei'][:]
    selected_channels = group['selected_channels'][:]
    
    peak_channel = group.attrs['peak_channel']

# Check shapes or values
print("Spike times:", spike_times.shape)
#print("EI shape:", ei.shape)
# print("Selected channels:", selected_channels)
# print(179 in selected_channels)
#print("Peak channel:", peak_channel)
print(spike_times)


unit_id = 11 # or whatever unit you're checking

with h5py.File(h5_out_path, 'r') as h5:
    group = h5[f'unit_{unit_id}']
    
    spike_times1 = group['spike_times'][:]
    ei = group['ei'][:]
    selected_channels = group['selected_channels'][:]
    
    peak_channel = group.attrs['peak_channel']

# Check shapes or values
print("Spike times:", spike_times1.shape)

### plot one channel

In [ ]:
# Plot
# ei = np.mean(snips_baselined, axis=2)
plt.figure(figsize=(7, 4))
plt.plot(ei[125,:], color='black', linewidth=1)
plt.plot(ei1[125,:], color='red', linewidth=1)
plt.xlabel("Time (samples)")
plt.ylabel("Amplitude")
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt


ref_channel = 51

# Define window around each spike
pre, post = 20, 60
snip_len = pre + post + 1

# Allocate array to hold all snippets
snippets = []

for s in spikes_1[:50]:
    if s - pre >= 0 and s + post < raw_data.shape[0]:
        snippet = raw_data[s - pre : s + post + 1, ref_channel]
        snippets.append(snippet)

snippets = np.array(snippets)  # shape: [n_spikes, snip_len]

# Plot all snippets
plt.figure(figsize=(10, 5))
for i in range(snippets.shape[0]):
    plt.plot(snippets[i], color='black', alpha=0.1, linewidth=0.5)


snippets = []

for s in unmatched_spikes_2[:50]:
    if s - pre >= 0 and s + post < raw_data.shape[0]:
        snippet = raw_data[s - pre : s + post + 1, ref_channel]
        snippets.append(snippet)

snippets = np.array(snippets)  # shape: [n_spikes, snip_len]

# Plot all snippets
for i in range(snippets.shape[0]):
    plt.plot(snippets[i], color='red', alpha=0.1, linewidth=1)

plt.title(f"Ref channel {ref_channel} waveforms at {len(snippets)} spikes")
plt.xlabel("Sample index (relative to spike)")
plt.ylabel("Amplitude")
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
snips, valid_spike_times = axolotl_utils_ram.extract_snippets_ram(
    raw_data=raw_data,
    spike_times=ks_spikes[mean_scores_at_spikes<20000],
    window=window,
    selected_channels=np.arange(512)
)

segment_len = 100_000
snips_baselined = snips.copy()  # shape (n_channels, 81, N)
n_channels, snip_len, n_spikes = snips_baselined.shape

# Determine segment index for each spike
segment_indices = valid_spike_times // segment_len  # shape: (n_spikes,)

# Loop through channels and subtract baseline per spike
for ch in range(n_channels):
    snips_baselined[ch, :, :] -= baselines[ch, segment_indices][None, :]


ei = np.mean(snips_baselined, axis=2)
#ei -= ei[:, :5].mean(axis=1, keepdims=True)

from plot_ei_waveforms import plot_ei_waveforms
import matplotlib.pyplot as plt

plt.figure(figsize=(15, 5))
plot_ei_waveforms(
    ei=ei,
    positions=ei_positions,
    scale=70.0,
    box_height=1.0,
    box_width=50,
    linewidth=0.5,
    alpha=0.9,
    colors='black'
)
plt.title("EI")
plt.tight_layout()
plt.show()

In [ ]:
import axolotl_utils_ram
import importlib
importlib.reload(axolotl_utils_ram)

(
spikes,
mean_score,
valid_score,
mean_scores_at_spikes,
valid_scores_at_spikes,
mean_thresh,
valid_thresh
) = axolotl_utils_ram.ei_pursuit_ram(
    raw_data=raw_data,
    spikes=ks_spikes,                     # absolute sample times
    ei_template=ei,                    # EI from selected cluster
    save_prefix='/Volumes/Lab/Users/alexth/axolotl/ei_scan_unit0',  # set uniquely per unit
    alignment_offset = -window[0],
    fit_percentile = 40,                # how many (percentile) spikes to take to fit Gaussian for threshold determination (left-hand side of already found spikes)
    sigma_thresh = 5.0,                  # how many Gaussian sigmas to take for threshold
    return_debug=True, 

)

In [ ]:
print(len(spikes))
ss=ks_spikes[mean_scores_at_spikes<20000]
print(len(ss))

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 5))
plt.hist(mean_scores_at_spikes, bins=200, alpha=0.5, label='KS spike scores', color='red')
plt.xlabel("Mean EI Match Score")
plt.ylabel("Count")
plt.title("Mean EI Scores: Global vs. KS-aligned")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
print(f"NaNs in scores: {np.isnan(scores).sum()}")


In [ ]:
from scipy.stats import norm
import numpy as np

fit_percentile = 40
sigma_thresh = 5.0

scores = mean_scores_at_spikes  # KS spike scores

clean_scores = mean_scores_at_spikes[~np.isnan(mean_scores_at_spikes)]

# 1. Determine percentile cutoff
cutoff = np.percentile(clean_scores, fit_percentile, method='nearest')
print(cutoff)

# 2. Select left tail
left_tail = clean_scores[clean_scores <= cutoff]

# 3. Fit normal distribution to tail
mu, sigma = norm.fit(left_tail)

# 4. Compute final threshold
threshold = mu - sigma_thresh * sigma

print(f"Fitted mu = {mu:.3f}, sigma = {sigma:.3f}")
print(f"Computed threshold = {threshold:.3f}")
import matplotlib.pyplot as plt

plt.figure(figsize=(8, 4))
plt.hist(left_tail, bins=100, density=True, alpha=0.5, label="Left tail")

# Overlay fitted Gaussian
x = np.linspace(left_tail.min(), left_tail.max(), 200)
pdf = norm.pdf(x, mu, sigma)
plt.plot(x, pdf, 'r-', label=f"Fit: μ={mu:.2f}, σ={sigma:.2f}")

plt.axvline(threshold, color='red', linestyle='--', label=f"Threshold = {threshold:.2f}")
plt.xlabel("Score")
plt.ylabel("Density")
plt.title("Fit to Left Tail of Scores (KS Spikes)")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
import h5py

unit_id_1 = 1000
unit_id_2 = 11
max_diff = 10 # in samples

# --- Load spike times ---
with h5py.File(h5_out_path, 'r') as h5:
    spikes_1 = h5[f'unit_{unit_id_1}']['spike_times'][:]
    spikes_2 = h5[f'unit_{unit_id_2}']['spike_times'][:]

# --- Sort spike times (for efficiency) ---
spikes_1 = np.sort(spikes_1)
spikes_2 = np.sort(spikes_2)

# --- Match spikes within max_diff ---
matches = []
j_start = 0

for i, t1 in enumerate(spikes_1):
    while j_start < len(spikes_2) and spikes_2[j_start] < t1 - max_diff:
        j_start += 1

    j = j_start
    while j < len(spikes_2) and spikes_2[j] <= t1 + max_diff:
        if abs(t1 - spikes_2[j]) <= max_diff:
            matches.append((i, j))
        j += 1

# --- Output ---
print(f"Found {len(matches)} matched spikes between unit {unit_id_1} and {unit_id_2}")
for i, j in matches:
    print(f"Spike1: {spikes_1[i]}, Spike2: {spikes_2[j]}")

# --- Build set of matched indices in spikes_2 ---
matched_j = set(j for _, j in matches)

# --- Extract unmatched spike times from spikes_2 ---
unmatched_spikes_2 = [spikes_2[j] for j in range(len(spikes_2)) if j not in matched_j]

print(f"Found {len(unmatched_spikes_2)} unmatched spikes in unit {unit_id_2}")
unmatched_spikes_2 = np.array(unmatched_spikes_2)


### plot EI

In [ ]:
from plot_ei_waveforms import plot_ei_waveforms
import matplotlib.pyplot as plt

plt.figure(figsize=(15, 5))
plot_ei_waveforms(
    ei=ei,
    positions=ei_positions,
    scale=70.0,
    box_height=1.0,
    box_width=50,
    linewidth=0.5,
    alpha=0.9,
    colors='black'
)
plt.title("EI")
plt.tight_layout()
plt.show()

In [ ]:
from plot_ei_waveforms import plot_ei_waveforms
import matplotlib.pyplot as plt

for i, cluster in enumerate(clusters_pre):
    ei = cluster['ei']
    ref_ch = cluster['channels'][np.argmax(np.ptp(ei[cluster['channels'], :], axis=1))]
    ei_p2p = np.ptp(ei[ref_ch, :])
    n_spikes = len(cluster['inds'])

    plt.figure(figsize=(15, 5))
    plot_ei_waveforms(
        ei=ei,
        positions=ei_positions,
        scale=70.0,
        box_height=1.0,
        box_width=50,
        linewidth=0.5,
        alpha=0.9,
        colors='black'
    )
    plt.title(f"Cluster {i} EI — Spikes: {n_spikes}, P2P on Ref Ch ({ref_ch}): {ei_p2p:.1f}")
    plt.tight_layout()
    plt.show()


In [ ]:
import axolotl_utils_ram
import importlib
importlib.reload(axolotl_utils_ram)

clusters_pre, pcs_pre, labels_pre, sim_matrix_pre, cluster_eis_pre  = axolotl_utils_ram.cluster_spike_waveforms(snips_baselined, ei, k_start=3,return_debug=True)


In [ ]:
# Plot
plt.figure(figsize=(7, 4))
plt.plot(cluster_eis_pre[2][148,:], color='black', linewidth=1)
plt.xlabel("Time (samples)")
plt.ylabel("Amplitude")
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:

ei_a = clusters_pre[0]['ei']
ei_b = clusters_pre[1]['ei']


result = axolotl_utils_ram.compare_ei_subtraction(ei_a, ei_b, max_lag=3, p2p_thresh=30.0)

res = np.array(result['per_channel_residuals'])
cos_sim = np.mean(result['per_channel_cosine_sim'])

neg_inds = np.where(res < -10)[0]

print(len(neg_inds))
print(cos_sim)



In [ ]:

ei_a = cluster_eis_pre[0]
ei_b = cluster_eis_pre[2]


result = axolotl_utils_ram.compare_ei_subtraction(ei_a, ei_b, max_lag=3, p2p_thresh=30.0)

res = np.array(result['per_channel_residuals'])
cos_sim = np.mean(result['per_channel_cosine_sim'])

neg_inds = np.where(res < -10)[0]

print(len(neg_inds))
print(cos_sim)


In [ ]:
from plot_ei_waveforms import plot_ei_waveforms
import matplotlib.pyplot as plt

plt.figure(figsize=(15, 5))
plot_ei_waveforms(
    ei=cluster_eis_pre[2],
    positions=ei_positions,
    scale=70.0,
    box_height=1.0,
    box_width=50,
    linewidth=0.5,
    alpha=0.9,
    colors='black'
)
plt.title(f"Cluster {i} EI — Spikes: {n_spikes}, P2P on Ref Ch ({ref_ch}): {ei_p2p:.1f}")
plt.tight_layout()
plt.show()

In [ ]:
# Plot
plt.figure(figsize=(7, 4))
plt.plot(ei_a[148,:], color='black', linewidth=1)
plt.plot(ei_b[148,:], color='red', linewidth=1)
plt.xlabel("Time (samples)")
plt.ylabel("Amplitude")
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:

global_cosine_sim = np.mean(result['per_channel_cosine_sim'])

amp_threshold = -10
cos_threshold = 0.9

res = result['per_channel_residuals']
neg_inds = np.where(np.array(res) < amp_threshold)[0]
if global_cosine_sim < cos_threshold or len(neg_inds) > 0:
    print('two units')
else:
    print('same unit')
    

In [ ]:

import matplotlib.pyplot as plt

plt.figure(figsize=(10, 4))
plt.bar(result['good_channels'], result['per_channel_residuals'], color='gray')
plt.axhline(0, color='black', linewidth=0.8, linestyle='--')
plt.xlabel("Channel ID")
plt.ylabel("Mean Residual (B - A)")
plt.title("Per-Channel Residuals (Masked Subtraction)")
plt.grid(True, axis='y')
plt.tight_layout()
plt.show()


plt.figure(figsize=(10, 4))
plt.bar(result['good_channels'], result['per_channel_cosine_sim'], color='gray')
plt.axhline(0, color='black', linewidth=0.8, linestyle='--')
plt.xlabel("Channel ID")
plt.ylabel("Mean Residual (B - A)")
plt.title("Per-Channel cosine_sim (Masked Subtraction)")
plt.grid(True, axis='y')
plt.tight_layout()
plt.show()

# plt.figure(figsize=(10, 4))
# plt.bar(result['good_channels'], result['p2p'], color='gray')
# plt.axhline(0, color='black', linewidth=0.8, linestyle='--')
# plt.xlabel("Channel ID")
# plt.ylabel("Mean Residual (B - A)")
# plt.title("Per-Channel amplitude")
# plt.grid(True, axis='y')
# plt.tight_layout()
# plt.show()


In [ ]:

    ei, spikes_idx, selected_channels, selected_cluster_index_pre = axolotl_utils_ram.select_cluster_with_largest_waveform(clusters_pre, ref_channel)

    spikes_init = spike_times[spikes_idx]

    if do_pursuit:
        (
        spikes,
        mean_score,
        valid_score,
        mean_scores_at_spikes,
        valid_scores_at_spikes,
        mean_thresh,
        valid_thresh
        ) = axolotl_utils_ram.ei_pursuit_ram(
            raw_data=raw_data,
            spikes=spikes_init,                     # absolute sample times
            ei_template=ei,                    # EI from selected cluster
            save_prefix='/Volumes/Lab/Users/alexth/axolotl/ei_scan_unit0',  # set uniquely per unit
            alignment_offset = -window[0],
            fit_percentile = 40,                # how many (percentile) spikes to take to fit Gaussian for threshold determination (left-hand side of already found spikes)
            sigma_thresh = 5.0,                  # how many Gaussian sigmas to take for threshold
            return_debug=True, 

        )
    else:
        spikes = spikes_init
        mean_score=None
        valid_score=None
        mean_scores_at_spikes=spikes
        valid_scores_at_spikes=None
        mean_thresh=None
        valid_thresh=None

    # Step 9a: Extract full snippets from final spike times

    snips_ref_channel, valid_spike_times = axolotl_utils_ram.extract_snippets_ram(
        raw_data=raw_data,
        spike_times=spikes,
        selected_channels=np.array([ref_channel]),
        window=window,
    )

    snips_ref_channel = snips_ref_channel.transpose(2, 0, 1)


    lags = axolotl_utils_ram.estimate_lags_by_xcorr_ram(
        snippets=snips_ref_channel,                # shape [N x C x T]
        peak_channel_idx=0,                 # 0 because the only channel that gets passed is the referent channel
        window=(-5, 10),                  # optional, relative to peak
        max_lag=6,                        # optional, max xcorr shift
    )

    spikes = spikes+lags

    snips_full, valid_spike_times = axolotl_utils_ram.extract_snippets_ram(
        raw_data=raw_data,
        spike_times=spikes,
        selected_channels=np.arange(n_channels),
        window=window,
    )


    segment_len = 100_000
    snips_baselined = snips_full.copy()  # shape (n_channels, 81, N)
    n_channels, snip_len, n_spikes = snips_baselined.shape

    # Determine segment index for each spike
    segment_indices = spikes // segment_len  # shape: (n_spikes,)

    # Loop through channels and subtract baseline per spike
    for ch in range(n_channels):
        snips_baselined[ch, :, :] -= baselines[ch, segment_indices][None, :]


    # Extract baseline-subtracted waveforms for ref_channel
    ref_snips = snips_baselined[ref_channel, :, :]  # shape: (81, N)

    # Mean waveform over all spikes
    ref_mean = ref_snips.mean(axis=1)  # shape: (81,)
    # Negative peak (should be near index 20)
    ref_peak_amp = np.abs(ref_mean[-window[0]])  # scalar

    # Threshold at 0.75× of mean waveform peak
    threshold_ampl = 0.75 * ref_peak_amp

    # Get all actual spike values at sample 20
    spike_amplitudes = np.abs(ref_snips[20, :])  # shape: (N,)

    # Flag bad spikes: too small
    bad_inds = np.where(spike_amplitudes < threshold_ampl)[0]

    # Create mask to keep only good spikes
    keep_mask = np.ones(spike_amplitudes.shape[0], dtype=bool)
    keep_mask[bad_inds] = False

    # --- Extract bad spike traces for plotting
    bad_spike_traces = snips_baselined[ref_channel, :, bad_inds]  # shape: (n_bad, T)

    # Get original traces for bad_spike_traces
    snips_bad = axolotl_utils_ram.extract_snippets_single_channel(
        dat_path='/Volumes/Lab/Users/alexth/axolotl/201703151_data001.dat',
        spike_times=spikes[bad_inds],
        ref_channel=ref_channel,
        window=window,
        n_channels=512,
        dtype='int16'
    )

    segment_indices = spikes[bad_inds] // segment_len  # shape: (n_spikes,)
    snips_bad[0, :, :] -= baselines[ref_channel, segment_indices][None, :]


    # Apply to real data and snips_baselined
    snips_baselined = snips_baselined[:, :, keep_mask]
    good_mean_trace = np.mean(snips_baselined[ref_channel, :, :], axis=1)
    snips_full = snips_full[:, :, keep_mask]
    valid_spike_times = valid_spike_times[keep_mask]
    spikes = spikes[keep_mask]

    spikes_for_plot_post = spikes

    final_spike_inds = np.where(keep_mask)[0]


        


In [ ]:
params = {
    'window': (-20, 60),
    'min_spikes': 100,
    'ei_sim_threshold': 0.75,
    'k_start': 4,
    'k_refine': 2
}

from verify_cluster import verify_cluster

spike_times = spikes
clusters = verify_cluster(
    spike_times=spike_times,
    dat_path=snips_baselined,
    params=params
)

print(f"Returned {len(clusters)} clean subclusters")
for i, cl in enumerate(clusters):
    print(f"  Cluster {i}: {len(cl['inds'])} spikes")

In [ ]:

import analyze_clusters
import importlib
importlib.reload(analyze_clusters)


analyze_clusters.analyze_clusters(clusters,
                 spike_times=spikes,
                 sampling_rate=20000,
                 dat_path=snips_baselined,
                 h5_path='/Volumes/Lab/Users/alexth/axolotl/201703151_kilosort_data001_spike_times.h5',
                 triggers_mat_path='/Volumes/Lab/Users/alexth/axolotl/trigger_in_samples_201703151.mat',
                 cluster_ids=None,
                 lut=None,
                 sta_depth=30,
                 sta_offset=0,
                 sta_chunk_size=1000,
                 sta_refresh=2,
                 ei_scale=3,
                 ei_cutoff=0.08)

In [ ]:
tmp = snips_baselined[ref_channel, 20, :].copy()
import matplotlib.pyplot as plt
plt.plot(tmp)

final_spike_inds = np.arange(len(spikes))


In [ ]:
plt.figure(figsize=(6, 4))
plt.hist(tmp, bins=50, color='gray', edgecolor='black')
plt.title("Histogram of tmp values")
plt.xlabel("Amplitude")
plt.ylabel("Count")
plt.grid(True)
plt.show()


In [ ]:
inds = np.where(tmp > -500)[0]
import matplotlib.pyplot as plt

plt.figure(figsize=(8, 4))
plt.plot(snips_baselined[39, :, inds].T, alpha=1)
plt.plot(snips_baselined[39, :, :6], alpha=1)
plt.title(f"Overlay of {len(inds)} selected snippets on channel 39")
plt.xlabel("Time (samples)")
plt.ylabel("Amplitude")
plt.grid(True)
plt.show()


In [ ]:


    # Step 9b: Recluster - choose k. snips_full is all channels, baselined - relevant cahnnels will be subselected in the function.


    if len(spikes)<100:
        pcs_post = np.zeros((1, 2))                    # shape: (N_spikes, 2 PCs)
        labels_post = np.array([0])                    # just one fake cluster label
        sim_matrix_post = np.zeros((1, 1))             # fake 1×1 similarity matrix
        ei_clusters_post = [np.zeros((512, 81))]       # fake EI for the “post” cluster
        selected_index_post = 0                        # only one cluster, so index is 0
        cluster_eis_post = [np.zeros((512, 81))]       # same dummy EI
        spikes_for_plot_post = np.array([0])           # placeholder spike time
        spike_counts_post = [len(snips)]               # use actual number of spikes
        matches = []                                # no matches
        # `snips_baselined` is [C x T x N]
        # We only subtract on the referent channel to avoid distortion
        template_fallback = np.mean(snips_baselined[ref_channel], axis=1)  # shape: (T,)
        residuals_fallback = snips_baselined[ref_channel] - template_fallback[:, None]  # shape: (T, N)

        # Assume residuals_fallback is (T, N) from previous step (template-subtracted waveforms)
        # Transpose to match expected shape: (n_spikes, snip_len)
        # force key and lookup to match normal case: np.int64
        ref_channel = np.int64(ref_channel)
        selected_channels = np.array([ref_channel], dtype=np.int64)
        residuals_per_channel = {
            ref_channel: residuals_fallback.T.astype(np.int16)
        }

    else:
        clusters_post, pcs_post, labels_post, sim_matrix_post, cluster_eis_post  = axolotl_utils_ram.cluster_spike_waveforms(snips=snips_baselined, ei=ei, k_start=2,return_debug=True)

        # Step 9c: choose the best cluster - choose similarity threshold. EI is all channels, baselined
        ei, final_spike_inds, selected_channels, selected_cluster_index_post = axolotl_utils_ram.select_cluster_by_ei_similarity_ram(clusters=clusters_post,reference_ei=ei,similarity_threshold=0.95)


        spikes = spikes[final_spike_inds]  # convert to absolute spike times
        snips_baselined = snips_baselined[:,:,final_spike_inds] # cut only the ones that survived

        p2p_threshold = 30
        ei_p2p = ei.max(axis=1) - ei.min(axis=1)
        selected_channels = np.where(ei_p2p > p2p_threshold)[0]
        selected_channels = selected_channels[np.argsort(ei_p2p[selected_channels])[::-1]]

        #print("reclustered pursuit\n")

        # check for matching KS units
        results = []
        lag = 20
        ks_sim_threshold = 0.75

        # Run comparison
        sim = compare_eis(ks_ei_stack, ei, lag).squeeze() # shape: (num_KS_units,)
        matches = [
            {
                "unit_id": ks_unit_ids[i],
                "vision_id": int(ks_vision_ids[ks_unit_ids[i]].item()),
                "similarity": float(sim[i]),
                "n_spikes": int(ks_n_spikes[ks_unit_ids[i]])
            }
            for i in np.where(sim > ks_sim_threshold)[0]
        ]



In [ ]:
p2p_threshold = 30
ei_p2p = ei.max(axis=1) - ei.min(axis=1)
selected_channels = np.where(ei_p2p > p2p_threshold)[0]
selected_channels = selected_channels[np.argsort(ei_p2p[selected_channels])[::-1]]

#print("reclustered pursuit\n")

# check for matching KS units
results = []
lag = 20
ks_sim_threshold = 0.75

# Run comparison
sim = compare_eis(ks_ei_stack, ei, lag).squeeze() # shape: (num_KS_units,)
matches = [
    {
        "unit_id": ks_unit_ids[i],
        "vision_id": int(ks_vision_ids[ks_unit_ids[i]].item()),
        "similarity": float(sim[i]),
        "n_spikes": int(ks_n_spikes[ks_unit_ids[i]])
    }
    for i in np.where(sim > ks_sim_threshold)[0]
]

pcs_post = np.zeros((1, 2))                    # shape: (N_spikes, 2 PCs)
labels_post = np.array([0])                    # just one fake cluster label
sim_matrix_post = np.zeros((1, 1))             # fake 1×1 similarity matrix
ei_clusters_post = [np.zeros((512, 81))]       # fake EI for the “post” cluster
selected_index_post = 0                        # only one cluster, so index is 0
cluster_eis_post = [np.zeros((512, 81))]       # same dummy EI
spikes_for_plot_post = np.array([0])           # placeholder spike time
spike_counts_post = [len(snips)]               # use actual number of spikes

In [ ]:


    # DIAGNOSTIC PLOTS

    axolotl_utils_ram.plot_unit_diagnostics(
        output_path=debug_folder,
        unit_id=unit_id,

        # --- From first call to cluster_spike_waveforms
        pcs_pre=pcs_pre,
        labels_pre=labels_pre,
        sim_matrix_pre=sim_matrix_pre,
        cluster_eis_pre = cluster_eis_pre,
        spikes_for_plot_pre = spikes_for_plot_pre,

        # --- From ei_pursuit
        mean_score=mean_score,
        valid_score=valid_score,
        mean_scores_at_spikes=mean_scores_at_spikes,
        valid_scores_at_spikes=valid_scores_at_spikes,
        mean_thresh=mean_thresh,
        valid_thresh=valid_thresh,

        # --- Lag estimation and bad spike filtering
        lags=lags,
        bad_spike_traces=bad_spike_traces,  # shape: (n_bad, T)
        good_mean_trace=good_mean_trace,
        threshold_ampl=-threshold_ampl,
        ref_channel=ref_channel,
        snips_bad=snips_bad,

        # --- From second clustering
        pcs_post=pcs_post,
        labels_post=labels_post,
        sim_matrix_post=sim_matrix_post,
        cluster_eis_post = cluster_eis_post,
        spikes_for_plot_post = spikes_for_plot_post,

        # --- For axis labels etc.
        window=(-20, 60),

        ei_positions=ei_positions,
        selected_channels_count=len(selected_channels),

        spikes = spikes, 
        orig_threshold = threshold,
        ks_matches = matches
    )


    # Step 10: Save unit metadata
    try:
        with h5py.File(h5_out_path, 'a') as h5:
            group = h5.require_group(f'unit_{unit_id}')

            for name, data in [
                ('spike_times', spikes.astype(np.int32)),
                ('ei', ei.astype(np.float32)), # EI is already baselined
                ('selected_channels', selected_channels.astype(np.int32))
            ]:
                if name in group:
                    del group[name]
                group.create_dataset(name, data=data)

            group.attrs['peak_channel'] = int(np.argmax(np.ptp(ei, axis=1)))
            # group.create_dataset('spike_times', data=spikes.astype(np.int32))
            # group.create_dataset('ei', data=ei.astype(np.float32))
            # group.create_dataset('selected_channels', data=selected_channels.astype(np.int32))
            # group.attrs['peak_channel'] = int(np.argmax(np.ptp(ei, axis=1)))

        #print(f"Exported unit_{unit_id} with {len(spikes)} spikes.")

    except KeyboardInterrupt:
        print("\nKeyboard interrupt detected — exiting safely before write completes.")

    except Exception as e:
        print(f"\nUnexpected error while saving unit_{unit_id}: {e}")



In [ ]:


    if len(spikes)>=100:
        snips_full = snips_full[np.ix_(selected_channels, np.arange(snips_full.shape[1]), final_spike_inds)]
        snips_full = snips_full.transpose(2, 0, 1) # [C × T × N] → [N × C × T]

            # --- Setup ---
        residuals_per_channel = {}
        cluster_ids_per_channel = {}
        scale_factors_per_channel = {}

        for ch_idx, ch in enumerate(selected_channels):
            # Slice data for this channel
            ch_snips = snips_full[:, ch_idx, :]  # shape: (n_spikes, snip_len)
            ch_baselines = baselines[ch, :]    # shape: (n_segments,)

            # Subtract PCA cluster means
            residuals, cluster_ids, scale_factors = axolotl_utils_ram.subtract_pca_cluster_means_ram(
                snippets=ch_snips,
                baselines=ch_baselines,
                spike_times=spikes,
                segment_len=100_000,  # must match what was used to generate baselines
                n_clusters=5,
                offset_window=(-10,40)
            )

            # Store results
            residuals_per_channel[ch] = residuals
            cluster_ids_per_channel[ch] = cluster_ids
            scale_factors_per_channel[ch] = scale_factors
    else:
        
        # We only subtract on the referent channel to avoid distortion
        template_fallback = np.mean(snips_baselined[ref_channel], axis=1)  # shape: (T,)
        residuals_fallback = snips_baselined[ref_channel] - template_fallback[:, None]  # shape: (T, N)

        # Assume residuals_fallback is (T, N) from previous step (template-subtracted waveforms)
        # Transpose to match expected shape: (n_spikes, snip_len)
        # force key and lookup to match normal case: np.int64
        ref_channel = np.int64(ref_channel)
        selected_channels = np.array([ref_channel], dtype=np.int64)
        residuals_per_channel = {
            ref_channel: residuals_fallback.T.astype(np.int16)
        }


    # end_time = time.time()
    # elapsed = end_time - start_time 
    # print(f"Finished preprocessing, starting edits. Elapsed: {elapsed:.1f} seconds.")
    # Step 12: edit raw data
    write_locs = spikes + window[0]
    axolotl_utils_ram.apply_residuals(
        raw_data=raw_data,
        dat_path = '/Volumes/Lab/Users/alexth/axolotl/201703151_data001_sub.dat',
        residual_snips_per_channel=residuals_per_channel,
        write_locs=write_locs,
        selected_channels=selected_channels,
        total_samples=raw_data.shape[0],
        dtype = np.int16,
        n_channels = n_channels,
        is_ram=True,
        is_disk=False
    )
    end_time = time.time()
    elapsed = end_time - start_time
    print(f"Processed unit {unit_id} with {len(spikes)} final spikes in {elapsed:.1f} seconds.\n")


    # Step 13: Repeat until done
    unit_id += 1
    # if unit_id >= max_units:
    #     print("Reached unit limit.")
    #     break

